<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">OC SWITRS GIS Data Processing</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 2 - Map Processing</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.1, January 2025</div>
</center></div>

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [371]:
#%reset -f

Instantiating python libraries for the project

In [1]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor

Which running environment this notebook is using? (1 = Visual Studio Code, 2 = ArcGIS Pro)

In [2]:
runEnv = input("Enter the running environment (1=VSCode, 2=ArcGIS Pro): ")

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

Define and maitnain project, workspace, ArcGIS, and data-related variables

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project and Geodatabase Paths</h3>

Define the ArcGIS pro project variables

In [3]:
# Environment variable for OneDrive path
onedriveFolder = os.getenv("OneDriveCommercial")

# OCSWITRS project path
projectFolder = os.path.join(onedriveFolder, "Documents", "OCSWITRS")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">ArcGIS Pro Paths</h3>

ArcGIS pro related paths

In [4]:
# OCSWITRS project AGP path
agpFolder = os.path.join(projectFolder, "AGPSWITRS")

# AGP APRX file name and path
aprxName = "AGPSWITRS.aprx"
aprxPath = os.path.join(agpFolder, aprxName)

# ArcGIS Pro project geodatabase and path
gdbName = "AGPSWITRS.gdb"
gdbPath = os.path.join(agpFolder, gdbName)

# ArcGIS pro project
if runEnv == "1":  # VSCode
    print("Running in VSCode (project = aprxPath)")
    aprx = arcpy.mp.ArcGISProject(aprxPath)
elif runEnv == "2":  # ArcGIS Pro
    print("Running in ArcGIS Pro (project = CURRENT)")
    aprx = arcpy.mp.ArcGISProject('CURRENT')

# Close all map views
aprx.closeViews()

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace
# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True
# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

Running in VSCode (project = aprxPath)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Folder Paths</h3>

In [5]:
# Raw data folder path
rawDataFolder = os.path.join(projectFolder, "Data", "Raw")

# Maps folder path
mapsFolder = os.path.join(projectFolder, "Maps")

# Layers folder path
layersFolder = os.path.join(projectFolder, "Layers")
layersTemplates = os.path.join(layersFolder, "Templates")

# Layouts folder path
layoutsFolder = os.path.join(projectFolder, "Layouts")

# Notebooks folder path
notebooksFolder = os.path.join(projectFolder, "Notebooks")
codebookPath = os.path.join(projectFolder, "Data", "codebook", "cb.json")

Geodatabase feature datasets paths (directories)

In [6]:
# RawData feature dataset in the geodatabase
gdbRawData = os.path.join(gdbPath, "RawData")

# RawData feature dataset in the geodatabase
gdbSupportingData = os.path.join(gdbPath, "SupportingData")

# AnalysisData feature dataset in the geodatabase
gdbAnalysisData = os.path.join(gdbPath, "AnalysisData")

# HotSpotData feature dataset in the geodatabase
gdbHotspotData = os.path.join(gdbPath, "HotSpotData")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Data Folder Paths</h3>

The most current raw data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [7]:
# Add the start date of the raw data to a new python datetime object
dateStart = datetime(2012, 1, 1, 0, 0, 0)

# Add the end date of the raw data to a new python datetime object
dateEnd = datetime(2024, 9, 30, 23, 59, 59)

# Define time and date variables
timeZone = pytz.timezone("US/Pacific")
today = datetime.now(timeZone)
dateUpdated = today.strftime("%B %d, %Y")
timeUpdated = today.strftime("%I:%M %p")

Define date strings for metadata

In [8]:
# String defining the years of the raw data
mdYears = f"{dateStart.year}-{dateEnd.year}"

# String defining the start and end dates of the raw data
mdDates = f"Data from {dateStart.strftime('%B %d, %Y')} to {dateEnd.strftime('%B %d, %Y')}"

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Codebook</h3>

In [9]:
# Load the JSON file from directory and store it in a variable
with open(codebookPath) as jsonFile:
    codebook = json.load(jsonFile)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">JSON CIM Exports</h3>

Creating a function to export the CIM JSON files to disk.

In [10]:
def exportCim(cimType, cimObject, cimName):
    """Export a CIM object to a file in both native (MAPX, PAGX, LYRX) and JSON CIM formats."""
    match cimType:
        # When the CIM object is a map
        case "map":
            # Export the CIM object to a MAPX file
            print(f"Exporting {cimName} map to MAPX...")
            cimObject.exportToMAPX(os.path.join(mapsFolder, cimName+".mapx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} map to JSON...\n")
            with open(os.path.join(mapsFolder, cimName+".mapx"), "r") as f:
                data = f.read()
            with open(os.path.join(mapsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layout
        case "layout":
            # Export the CIM object to a PAGX file
            print(f"Exporting {cimName} layout to PAGX...")
            cimObject.exportToPAGX(os.path.join(layoutsFolder, cimName+".pagx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layout to JSON...\n")
            with open(os.path.join(layoutsFolder, cimName+".pagx"), "r") as f:
                data = f.read()
            with open(os.path.join(layoutsFolder, cimName+".json"), "w") as f:
                f.write(data)
        
        # When the CIM object is a layer
        case "layer":
            # Export the CIM object to a LYRX file
            print(f"Exporting {cimName} layer to LYRX...")
            # Reformat the name of the output file
            for m in aprx.listMaps():
                for l in m.listLayers():
                    if l == cimObject:
                        cimNewName = m.name.title()+"Map-"+l.name.replace("OCSWITRS ", "")
            # Save the layer to a LYRX file
            arcpy.management.SaveToLayerFile(cimObject, os.path.join(layersFolder, cimNewName + ".lyrx"))
            print(arcpy.GetMessages())
            
            # Export the CIM object to a JSON file
            print(f"Exporting {cimName} layer to JSON...\n")
            with open(os.path.join(layersFolder, cimNewName+".lyrx"), "r") as f:
                data = f.read()
            with open(os.path.join(layersFolder, cimNewName+".json"), "w") as f:
                f.write(data)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.3. ArcGIS Pro Workspace</h2>

Set the workspace and environment settings for the ArcGIS Pro project

In [11]:
# Set the workspace and environment to the root of the project geodatabase
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True

# Disable adding outputs to map
arcpy.env.addOutputsToMap = False

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.4. Map List</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Maps</h3>

In [12]:
# List of maps to be created for the project
mapList = ["collisions", "crashes", "parties", "victims", "injuries", "fatalities", "fhs100m1km", "fhs150m2km", "fhs100m5km", "fhsRoads500ft", "ohsRoads500ft", "roadCrashes",
           "roadHotspots", "roadBuffers", "roadSegments", "roads", "pointFhs", "pointOhs", "popDens", "houDens", "areaCities", "areaBlocks", "summaries", "analysis", "regression"]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Project Layouts</h3>

In [13]:
# List or layouts to be created for the project
layoutList = ["maps", "injuries", "hotspots", "roads", "points", "densities", "areas"]

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.5. Feature Class Definitions</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Raw Data Feature Classes</h3>

Define paths for the feature classes in the raw data feature dateset of the geodatabase

In [14]:
collisions = os.path.join(gdbRawData, "collisions")
crashes = os.path.join(gdbRawData, "crashes")
parties = os.path.join(gdbRawData, "parties")
victims = os.path.join(gdbRawData, "victims")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Supporting Data Feature Classes</h3>

Define paths for the feature classes in the supporting data feature dateset of the geodatabase

In [15]:
boundaries = os.path.join(gdbSupportingData, "boundaries")
cities = os.path.join(gdbSupportingData, "cities")
censusBlocks = os.path.join(gdbSupportingData, "censusBlocks")
roads = os.path.join(gdbSupportingData, "roads")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Analysis Data Feature Classes</h3>

Define paths for the feature classes in the analysis data feature dateset of the geodatabase

In [16]:
roadsMajor = os.path.join(gdbAnalysisData, "roadsMajor")
roadsMajorBuffers = os.path.join(gdbAnalysisData, "roadsMajorBuffers")
roadsMajorBuffersSum = os.path.join(gdbAnalysisData, "roadsMajorBuffersSum")
roadsMajorPointsAlongLines = os.path.join(gdbAnalysisData, "roadsMajorPointsAlongLines")
roadsMajorSplit = os.path.join(gdbAnalysisData, "roadsMajorSplit")
roadsMajorSplitBuffer = os.path.join(gdbAnalysisData, "roadsMajorSplitBuffer")
roadsMajorSplitBufferSum = os.path.join(gdbAnalysisData, "roadsMajorSplitBufferSum")
censusBlocksSum = os.path.join(gdbAnalysisData, "censusBlocksSum")
citiesSum = os.path.join(gdbAnalysisData, "citiesSum")
crashes500ftFromMajorRoads = os.path.join(gdbAnalysisData, "crashes500ftFromMajorRoads")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Hot Spot Data Feature Classes</h3>

Define paths for the feature classes in the hot spot data feature dateset of the geodatabase

In [17]:
crashesHotspots = os.path.join(gdbHotspotData, "crashesHotspots")
crashesOptimizedHotspots = os.path.join(gdbHotspotData, "crashesOptimizedHotspots")
crashesFindHotspots100m1km = os.path.join(gdbHotspotData, "crashesFindHotspots100m1km")
crashesFindHotspots150m2km = os.path.join(gdbHotspotData, "crashesFindHotspots150m2km")
crashesFindHotspots100m5km = os.path.join(gdbHotspotData, "crashesFindHotspots100m5km")
crashesHotspots500ftFromMajorRoads = os.path.join(gdbHotspotData, "crashesHotspots500ftFromMajorRoads")
crashesFindHotspots500ftMajorRoads500ft1mi = os.path.join(gdbHotspotData, "crashesFindHotspots500ftMajorRoads500ft1mi")

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">2. Project Maps</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.1. Setup Maps</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Remove Old Maps</h3>

First delete all raw data maps from ArcGIS pro current project

In [18]:
if aprx.listMaps():
    for m in aprx.listMaps():
        if m.name in mapList:
            print(f"Removing {m.name} map from the project...")
            aprx.deleteItem(m)
        else:
            print(f"Map {m.name} is not in the list of maps to be created.")
else:
    print("No maps are currently in the project.")

Removing analysis map from the project...
Removing areaBlocks map from the project...
Removing areaCities map from the project...
Removing collisions map from the project...
Removing crashes map from the project...
Removing fatalities map from the project...
Removing fhs100m1km map from the project...
Removing fhs100m5km map from the project...
Removing fhs150m2km map from the project...
Removing fhsRoads500ft map from the project...
Removing houDens map from the project...
Removing injuries map from the project...
Removing ohsRoads500ft map from the project...
Removing parties map from the project...
Removing pointFhs map from the project...
Removing pointOhs map from the project...
Removing popDens map from the project...
Removing regression map from the project...
Removing roadBuffers map from the project...
Removing roadCrashes map from the project...
Removing roadHotspots map from the project...
Removing roads map from the project...
Removing roadSegments map from the project...
R

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Create New Maps</h3>

Create new raw data maps in current ArcGIS Pro project

In [19]:
# for each of the maps in the list, if it exists, delete it
c = 1
for m in mapList:
    for i in aprx.listMaps():
        if i.name == m:
            print(f"Deleting map: {m}")
            aprx.deleteItem(i)
    # Create new maps
    print(f"Creating map {c}: {m}")
    aprx.createMap(m)
    c += 1

Creating map 1: collisions
Creating map 2: crashes
Creating map 3: parties
Creating map 4: victims
Creating map 5: injuries
Creating map 6: fatalities
Creating map 7: fhs100m1km
Creating map 8: fhs150m2km
Creating map 9: fhs100m5km
Creating map 10: fhsRoads500ft
Creating map 11: ohsRoads500ft
Creating map 12: roadCrashes
Creating map 13: roadHotspots
Creating map 14: roadBuffers
Creating map 15: roadSegments
Creating map 16: roads
Creating map 17: pointFhs
Creating map 18: pointOhs
Creating map 19: popDens
Creating map 20: houDens
Creating map 21: areaCities
Creating map 22: areaBlocks
Creating map 23: summaries
Creating map 24: analysis
Creating map 25: regression


Store the map objects in variables

In [20]:
# OCSWITRS Data Maps
mapCollisions = aprx.listMaps("collisions")[0]
mapCrashes = aprx.listMaps("crashes")[0]
mapParties = aprx.listMaps("parties")[0]
mapVictims = aprx.listMaps("victims")[0]
mapInjuries = aprx.listMaps("injuries")[0]
mapFatalities = aprx.listMaps("fatalities")[0]

# OCSWITRS Hotspot Maps
mapFhs100m1km = aprx.listMaps("fhs100m1km")[0]
mapFhs150m2km = aprx.listMaps("fhs150m2km")[0]
mapFhs100m5km = aprx.listMaps("fhs100m5km")[0]
mapFhsRoads500ft = aprx.listMaps("fhsRoads500ft")[0]
mapOhsRoads500ft = aprx.listMaps("ohsRoads500ft")[0]
mapPointFhs = aprx.listMaps("pointFhs")[0]
mapPointOhs = aprx.listMaps("pointOhs")[0]

# OCSWITRS Supporting Data Maps
mapRoads = aprx.listMaps("roads")[0]
mapRoadCrashes = aprx.listMaps("roadCrashes")[0]
mapRoadHotspots = aprx.listMaps("roadHotspots")[0]
mapRoadBuffers = aprx.listMaps("roadBuffers")[0]
mapRoadSegments = aprx.listMaps("roadSegments")[0]
mapPopDens = aprx.listMaps("popDens")[0]
mapHouDens = aprx.listMaps("houDens")[0]
mapAreaCities = aprx.listMaps("areaCities")[0]
mapAreaBlocks = aprx.listMaps("areaBlocks")[0]

# OCSWITRS Analysis and Processing Maps
mapSummaries = aprx.listMaps("summaries")[0]
mapAnalysis = aprx.listMaps("analysis")[0]
mapRegression = aprx.listMaps("regression")[0]

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Change Map Basemaps</h3>

For each map, replace the existing basemap ("Topographic") with the "Dark Gray Canvas" basemap

In [21]:
for m in aprx.listMaps():
    print(f"Map: {m.name}")
    for l in m.listLayers():
        if l.isBasemapLayer:
            print(f"  - Removing Basemap: {l.name}")
            m.removeLayer(l)
    print(f"  - Adding Basemap: Light Gray Canvas")
    m.addBasemap("Light Gray Canvas")

Map: analysis
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: areaBlocks
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: areaCities
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: collisions
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: crashes
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: fatalities
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: fhs100m1km
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: fhs100m5km
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: fhs150m2km
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: fhsRoads500ft
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: houDens
  - Removing Basemap: Topographic
  - Adding Basemap: Light Gray Canvas
Map: injuries
  - Removing Basemap: T

Turn off the basemap reference layer for all maps

In [22]:
for m in aprx.listMaps():
    print(f"Map: {m.name}")
    for l in m.listLayers():
        if l.name == "Light Gray Reference":
            l.visible = False

Map: analysis
Map: areaBlocks
Map: areaCities
Map: collisions
Map: crashes
Map: fatalities
Map: fhs100m1km
Map: fhs100m5km
Map: fhs150m2km
Map: fhsRoads500ft
Map: houDens
Map: injuries
Map: ohsRoads500ft
Map: parties
Map: pointFhs
Map: pointOhs
Map: popDens
Map: regression
Map: roadBuffers
Map: roadCrashes
Map: roadHotspots
Map: roads
Map: roadSegments
Map: summaries
Map: victims


<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.2. Map Metadata</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">1. Collisions Map Metadata</h3>

Create a new metadata object for the collisions map and assign it to the map

In [23]:
mdoMapCollisions = md.Metadata()
mdoMapCollisions.title = "OCSWITRS Collisions Map"
mdoMapCollisions.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transporation"
mdoMapCollisions.summary = f"Statewide Integrated Traffic Records System (SWITRS) Collisions Map for Orange County, California ({mdYears})"
mdoMapCollisions.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapCollisions.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapCollisions.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapCollisions.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapCollisions.metadata = mdoMapCollisions

<h3 style="font-weight:bold; color:lime; padding-left: 50px">2. Crashes Map Metadata</h3>

Create a new metadata object for the crashes map and assign it to the map

In [24]:
mdoMapCrashes = md.Metadata()
mdoMapCrashes.title = "OCSWITRS Crashes Map"
mdoMapCrashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapCrashes.summary = f"Statewide Integrated Traffic Records System (SWITRS) Crashes Map for Orange County, California ({mdYears})"
mdoMapCrashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on crashes</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapCrashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapCrashes.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapCrashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapCrashes.metadata = mdoMapCrashes

<h3 style="font-weight:bold; color:lime; padding-left: 50px">3. Parties Map Metadata</h3>

Create a new metadata object for the parties map and assign it to the map

In [25]:
mdoMapParties = md.Metadata()
mdoMapParties.title = "OCSWITRS Parties Map"
mdoMapParties.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Parties, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapParties.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Parties Data for Orange County, California ({mdYears})"
mdoMapParties.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on parties involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapParties_credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapParties.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapParties.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/1e07bb1002f9457fa6fd3540fdb08e29/data"

# Apply the metadata to the map
mapParties.metadata = mdoMapParties

<h3 style="font-weight:bold; color:lime; padding-left: 50px">4. Victims Map Metadata</h3>

Create a new metadata object for the victims map and assign it to the map

In [26]:
mdoMapVictims = md.Metadata()
mdoMapVictims.title = "OCSWITRS Victims Map"
mdoMapVictims.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Victims, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapVictims.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Victims Data for Orange County, California ({mdYears})"
mdoMapVictims.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on victims/persons involved in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapVictims.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapVictims.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapVictims.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"

# Apply the metadata to the map
mapVictims.metadata = mdoMapVictims

<h3 style="font-weight:bold; color:lime; padding-left: 50px">5. Injuries Map Metadata</h3>

Create a new metadata object for the injuries map and assign it to the map

In [27]:
mdoMapInjuries = md.Metadata()
mdoMapInjuries.title = "OCSWITRS Injuries Map"
mdoMapInjuries.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Injuries, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapInjuries.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Injuries Data for Orange County, California ({mdYears})"
mdoMapInjuries.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on injuries sustained by victims in crash incidents</span><span> in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapInjuries.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapInjuries.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapInjuries.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"

# Apply the metadata to the map
mapInjuries.metadata = mdoMapInjuries

<h3 style="font-weight:bold; color:lime; padding-left: 50px">6. Fatalities Map Metadata</h3>

Create a new metadata object for the fatalities map and assign it to the map

In [28]:
mdoMapFatalities = md.Metadata()
mdoMapFatalities.title = "OCSWITRS Fatalities Map"
mdoMapFatalities.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transporation"
mdoMapFatalities.summary = f"Statewide Integrated Traffic Records System (SWITRS) Fatalities Map for Orange County, California ({mdYears})"
mdoMapFatalities.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">victim fatality counts per accident in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapFatalities.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapFatalities.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapFatalities.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapFatalities.metadata = mdoMapFatalities

<h3 style="font-weight:bold; color:lime; padding-left: 50px">7. Hotspots (100m, 1km) Map Metadata</h3>

Create a new metadata object for the hotspots (100m, 1km) map and assign it to the map

In [29]:
mdoMapFhs100m1km = md.Metadata()
mdoMapFhs100m1km.title = "OCSWITRS Hot Spot Analysis 100m 1km Map"
mdoMapFhs100m1km.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapFhs100m1km.summary = f"Statewide Integrated Traffic Records System (SWITRS) Hot Spot Analysis 100m 1km Map for Orange County, California ({mdYears})"
mdoMapFhs100m1km.description = """<div style="text-align:Left;"><div><div><p><span>Hot Spot Analysis for the OCSWITRS Project Data using 100m bins and 1km neighborhood radius grid</span></p></div></div></div>"""
mdoMapFhs100m1km.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapFhs100m1km.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapFhs100m1km.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapFhs100m1km.metadata = mdoMapFhs100m1km

<h3 style="font-weight:bold; color:lime; padding-left: 50px">8. Hotspots (150m, 2km) Map Metadata</h3>

Create a new metadata object for the hotspots (150m, 2km) map and assign it to the map

In [30]:
mdoMapFhs150m2km = md.Metadata()
mdoMapFhs150m2km.title = "OCSWITRS Hot Spot Analysis 150m 2km Map"
mdoMapFhs150m2km.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapFhs150m2km.summary = f"Statewide Integrated Traffic Records System (SWITRS) Hot Spot Analysis 150m 2km Map for Orange County, California ({mdYears})"
mdoMapFhs150m2km.description = """<div style="text-align:Left;"><div><div><p><span>Hot Spot Analysis for the OCSWITRS Project Data using 150m bins and 2km neighborhood radius grid</span></p></div></div></div>"""
mdoMapFhs150m2km.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapFhs150m2km.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapFhs150m2km.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapFhs150m2km.metadata = mdoMapFhs150m2km

<h3 style="font-weight:bold; color:lime; padding-left: 50px">9. Hotspots (100m, 5km) Map Metadata</h3>

Create a new metadata object for the hotspots (100m, 5km) map and assign it to the map

In [31]:
mdoMapFhs100m5km = md.Metadata()
mdoMapFhs100m5km.title = "OCSWITRS Hot Spot Analysis 100m 5km Map"
mdoMapFhs100m5km.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapFhs100m5km.summary = f"Statewide Integrated Traffic Records System (SWITRS) Hot Spot Analysis 100m 5km Map for Orange County, California ({mdYears})"
mdoMapFhs100m5km.description = """<div style="text-align:Left;"><div><div><p><span>Hot Spot Analysis for the OCSWITRS Project Data using 100m bins and 5km neighborhood radius grid</span></p></div></div></div>"""
mdoMapFhs100m5km.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapFhs100m5km.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapFhs100m5km.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapFhs100m5km.metadata = mdoMapFhs100m5km

<h3 style="font-weight:bold; color:lime; padding-left: 50px">10. Hotspots 500ft from Major Roads Map Metadata</h3>

Create a new metadata object for the hotspots 500ft from major roads map and assign it to the map

In [32]:
mdoMapFhsRoads500ft = md.Metadata()
mdoMapFhsRoads500ft.title = "OCSWITRS Hot Spots within 500ft from Major Roads Map"
mdoMapFhsRoads500ft.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapFhsRoads500ft.summary = f"Statewide Integrated Traffic Records System (SWITRS) Hot Spots within 500ft from Major Roads Map for Orange County, California ({mdYears})"
mdoMapFhsRoads500ft.description = """<div style="text-align:Left;"><div><div><p><span>Hot Spot Analysis for the OCSWITRS Project Data within 500ft from Major Roads</span></p></div></div></div>"""
mdoMapFhsRoads500ft.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapFhsRoads500ft.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapFhsRoads500ft.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapFhsRoads500ft.metadata = mdoMapFhsRoads500ft

<h3 style="font-weight:bold; color:lime; padding-left: 50px">11. Optimized Hotspots 500ft from Major Roads Map Metadata</h3>

Create a new metadata object for the optimized hotspots 500ft from major roads map and assign it to the map

In [33]:
mdoMapOhsRoads500ft = md.Metadata()
mdoMapOhsRoads500ft.title = "OCSWITRS Optimized Hot Spots within 500ft from Major Roads Map"
mdoMapOhsRoads500ft.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapOhsRoads500ft.summary = f"Statewide Integrated Traffic Records System (SWITRS) Optimized Hot Spots within 500ft from Major Roads Map for Orange County, California ({mdYears})"
mdoMapOhsRoads500ft.description = """<div style="text-align:Left;"><div><div><p><span>Optimized Hot Spot Analysis for the OCSWITRS Project Data within 500ft from Major Roads</span></p></div></div></div>"""
mdoMapOhsRoads500ft.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapOhsRoads500ft.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapOhsRoads500ft.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapOhsRoads500ft.metadata = mdoMapOhsRoads500ft

<h3 style="font-weight:bold; color:lime; padding-left: 50px">12. Major Road Crashes Map Metadata</h3>

Create a new metadata object for the major road crashes map and assign it to the map

In [34]:
mdoMapRoadCrashes = md.Metadata()
mdoMapRoadCrashes.title = "OCSWITRS Major Road Crashes Map"
mdoMapRoadCrashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapRoadCrashes.summary = f"Statewide Integrated Traffic Records System (SWITRS) Road Crashes Map for Orange County, California ({mdYears})"
mdoMapRoadCrashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on road crashes in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapRoadCrashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRoadCrashes.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRoadCrashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapRoadCrashes.metadata = mdoMapRoadCrashes

<h3 style="font-weight:bold; color:lime; padding-left: 50px">13. Major Road Hotspots Map Metadata</h3>

Create a new metadata object for the major road hotspots map and assign it to the map

In [35]:
mdoMapRoadHotspots = md.Metadata()
mdoMapRoadHotspots.title = "OCSWITRS Major Road Hotspots Map"
mdoMapRoadHotspots.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapRoadHotspots.summary = f"Statewide Integrated Traffic Records System (SWITRS) Road Hotspots Map for Orange County, California ({mdYears})"
mdoMapRoadHotspots.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on road hotspots in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapRoadHotspots.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRoadHotspots.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRoadHotspots.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapRoadHotspots.metadata = mdoMapRoadHotspots

<h3 style="font-weight:bold; color:lime; padding-left: 50px">14. Major Road Buffers Map Metadata</h3>

Create a new metadata object for the major road buffers map and assign it to the map

In [36]:
mdoMapRoadBuffers = md.Metadata()
mdoMapRoadBuffers.title = "OCSWITRS Major Road Buffers Map"
mdoMapRoadBuffers.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapRoadBuffers.summary = f"Statewide Integrated Traffic Records System (SWITRS) Road Buffers Map for Orange County, California ({mdYears})"
mdoMapRoadBuffers.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on road buffers in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapRoadBuffers.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRoadBuffers.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRoadBuffers.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapRoadBuffers.metadata = mdoMapRoadBuffers

<h3 style="font-weight:bold; color:lime; padding-left: 50px">15. Major Road Buffer Segments Map Metadata</h3>

Create a new metadata object for the major road buffer segments map and assign it to the map

In [37]:
mdoMapRoadSegments = md.Metadata()
mdoMapRoadSegments.title = "OCSWITRS Major Road Segments Map"
mdoMapRoadSegments.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapRoadSegments.summary = f"Statewide Integrated Traffic Records System (SWITRS) Road Segments Map for Orange County, California ({mdYears})"
mdoMapRoadSegments.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on road segments in Orange County, California for {mdYears} ({mdDates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{dateUpdated}</b></span></p></div></div></div>"""
mdoMapRoadSegments.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRoadSegments.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRoadSegments.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapRoadSegments.metadata = mdoMapRoadSegments

<h3 style="font-weight:bold; color:lime; padding-left: 50px">16. Roads Map Metadata</h3>

Create a new metadata object for the roads map and assign it to the map

In [38]:
mdoMapRoads = md.Metadata()
mdoMapRoads.title = "OCSWITRS Roads Processing Map"
mdoMapRoads.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoMapRoads.summary = "Roads Processing Map for the OCSWITRS Project"
mdoMapRoads.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County Roads Network is a comprehensive representation of all roads in the area, including primary roads and highways, secondary roads, and local roads. The data are sourced from the Orange County Department of Public Works and are updated regularly to reflect the most current road network configuration.</span></p></div></div></div>"""
mdoMapRoads.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRoads.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRoads.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

# Apply the metadata to the map
mapRoads.metadata = mdoMapRoads

<h3 style="font-weight:bold; color:lime; padding-left: 50px">17. Hotspot Points Map Metadata</h3>

Create a new metadata object for the hotspot points map and assign it to the map

In [39]:
mdoMapPointFhs = md.Metadata()
mdoMapPointFhs.title = "OCSWITRS Point Features Hot Spots Map"
mdoMapPointFhs.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapPointFhs.summary = f"Statewide Integrated Traffic Records System (SWITRS) Point Features Hot Spots Map for Orange County, California ({mdYears})"
mdoMapPointFhs.description = """<div style="text-align:Left;"><div><div><p><span>Hot Spot Analysis for the OCSWITRS Project Data using point features</span></p></div></div></div>"""
mdoMapPointFhs.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapPointFhs.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapPointFhs.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapPointFhs.metadata = mdoMapPointFhs

<h3 style="font-weight:bold; color:lime; padding-left: 50px">18. Optimized Hotspot Points Map Metadata</h3>

Create a new metadata object for the optimized hotspot points map and assign it to the map

In [40]:
mdoMapPointOhs = md.Metadata()
mdoMapPointOhs.title = "OCSWITRS Point Features Optimized Hot Spots Map"
mdoMapPointOhs.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapPointOhs.summary = f"Statewide Integrated Traffic Records System (SWITRS) Point Features Optimized Hot Spots Map for Orange County, California ({mdYears})"
mdoMapPointOhs.description = """<div style="text-align:Left;"><div><div><p><span>Optimized Hot Spot Analysis for the OCSWITRS Project Data using point features</span></p></div></div></div>"""
mdoMapPointOhs.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapPointOhs.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapPointOhs.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapPointOhs.metadata = mdoMapPointOhs

<h3 style="font-weight:bold; color:lime; padding-left: 50px">19. Population Density Map Metadata</h3>

Create a new metadata object for the population density map and assign it to the map

In [41]:
mdoMapPopDens = md.Metadata()
mdoMapPopDens.title = "OCSWITRS Population Density Map"
mdoMapPopDens.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoMapPopDens.summary = "Population Density Map for the OCSWITRS Project"
mdoMapPopDens.description = """<div style="text-align:Left;"><div><div><p><span>The map displays the population density (per square mile) in Orange County by Census Blocks (US Census 2000). The data source is the US Centennial Census of 2000.</span></p></div></div></div>"""
mdoMapPopDens.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapPopDens.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapPopDens.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

# Apply the metadata to the map
mapPopDens.metadata = mdoMapPopDens

<h3 style="font-weight:bold; color:lime; padding-left: 50px">20. Housing Density Map Metadata</h3>

Create a new metadata object for the housing density map and assign it to the map

In [42]:
mdoMapHouDens = md.Metadata()
mdoMapHouDens.title = "OCSWITRS Housing Density Map"
mdoMapHouDens.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoMapHouDens.summary = "Housing Density Map for the OCSWITRS Project"
mdoMapHouDens.description = """<div style="text-align:Left;"><div><div><p><span>The map displays the housing density (per square mile) in Orange County by Census Blocks (US Census 2000). The data source is the US Centennial Census of 2000.</span></p></div></div></div>"""
mdoMapHouDens.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapHouDens.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapHouDens.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

# Apply the metadata to the map
mapHouDens.metadata = mdoMapHouDens

<h3 style="font-weight:bold; color:lime; padding-left: 50px">21. City Areas Victims Map Metadata</h3>

Create a new metadata object for the victims by city areas map and assign it to the map

In [43]:
mdoMapAreaCities = md.Metadata()
mdoMapAreaCities.title = "OCSWITRS City Areas Victims Count Map"
mdoMapAreaCities.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoMapAreaCities.summary = "City Areas Victims Count Map for the OCSWITRS Project"
mdoMapAreaCities.description = """<div style="text-align:Left;"><div><div><p><span>The map displays the number of victims involved in traffic incidents in Orange County by City Areas. The data source is the SWITRS database maintained by the California Highway Patrol (CHP).</span></p></div></div></div>"""
mdoMapAreaCities.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapAreaCities.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapAreaCities.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

# Apply the metadata to the map
mapAreaCities.metadata = mdoMapAreaCities

<h3 style="font-weight:bold; color:lime; padding-left: 50px">22. Census Blocks Victims Map Metadata</h3>

Create a new metadata object for the victims by census blocks map and assign it to the map

In [44]:
mdoMapAreaBlocks = md.Metadata()
mdoMapAreaBlocks.title = "OCSWITRS Census Blocks Victims Count Map"
mdoMapAreaBlocks.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdoMapAreaBlocks.summary = "Census Blocks Victims Count Map for the OCSWITRS Project"
mdoMapAreaBlocks.description = """<div style="text-align:Left;"><div><div><p><span>The map displays the number of victims involved in traffic incidents in Orange County by Census Blocks. The data source is the SWITRS database maintained by the California Highway Patrol (CHP).</span></p></div></div></div>"""
mdoMapAreaBlocks.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapAreaBlocks.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapAreaBlocks.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"

# Apply the metadata to the map
mapAreaBlocks.metadata = mdoMapAreaBlocks

<h3 style="font-weight:bold; color:lime; padding-left: 50px">23. Summaries Map Metadata</h3>

Create a new metadata object for the summaries map and assign it to the map

In [45]:
mdoMapSummaries = md.Metadata()
mdoMapSummaries.title = "OCSWITRS Summaries Map"
mdoMapSummaries.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapSummaries.summary = f"Statewide Integrated Traffic Records System (SWITRS) Summaries Map for Orange County, California ({mdYears})"
mdoMapSummaries.description = """<div style="text-align:Left;"><div><div><p><span>Summarized data representation and visualization for the Orange County OCSWITRS traffic incident data.</span></p></div></div></div>"""
mdoMapSummaries.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapSummaries.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapSummaries.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapSummaries.metadata = mdoMapSummaries

<h3 style="font-weight:bold; color:lime; padding-left: 50px">24. Analysis Map Metadata</h3>

Create a new metadata object for the analysis map and assign it to the map

In [46]:
mdoMapAnalysis = md.Metadata()
mdoMapAnalysis.title = "OCSWITRS Analysis Map"
mdoMapAnalysis.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapAnalysis.summary = f"Statewide Integrated Traffic Records System (SWITRS) Analysis Map for Orange County, California ({mdYears})"
mdoMapAnalysis.description = """<div style="text-align:Left;"><div><div><p><span>Analysis and visualization for the OCSWITRS Project Data</span></p></div></div></div>"""
mdoMapAnalysis.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapAnalysis.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapAnalysis.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapAnalysis.metadata = mdoMapAnalysis

<h3 style="font-weight:bold; color:lime; padding-left: 50px">25. Regression Map Metadata</h3>

Create a new metadata object for the regression map and assign it to the map

In [47]:
mdoMapRegression = md.Metadata()
mdoMapRegression.title = "OCSWITRS Regression Analysis Map"
mdoMapRegression.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoMapRegression.summary = f"Statewide Integrated Traffic Records System (SWITRS) Regression Analysis Map for Orange County, California ({mdYears})"
mdoMapRegression.description = """<div style="text-align:Left;"><div><div><p><span>Regression Analysis for the OCSWITRS Project Data</span></p></div></div></div>"""
mdoMapRegression.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoMapRegression.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoMapRegression.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

# Apply the metadata to the map
mapRegression.metadata = mdoMapRegression

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [48]:
# Save the project
aprx.save()

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">3. Map Layers Processing</h1>

In this section we will be creating map layers for the feature classes in the geodatabase. The layers will be added to the maps and layouts.

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.1. Time Settings Configuration</h2>

Define time settings configuration for the map layers

In [49]:
# Define the key time parameters for the layers using a dictionary
timeSettings = {
    "st": datetime(2012, 1, 1, 0, 0, 0),
    "et": datetime(2024, 9, 30, 23, 59, 59),
    "td": datetime(2024, 9, 30, 23, 59, 59) - datetime(2012, 1, 1, 0, 0, 0),
    "stf": "dateDatetime",
    "tsi": 1.0,
    "tsiu": "months",
    "tz": arcpy.mp.ListTimeZones("*Pacific*")[0]
}
# where, st: start time, et: end time, td: time extent, stf: time field,
# tsi: time interval, tsiu: time units, tz: time zone

Define a function that enables time settings in data layers

In [50]:
def setLayerTime(layer):
    """Set the time properties for a layer in the map"""
    # Check if the layer is time-enabled
    if not layer.isTimeEnabled:
        # Enable time for the layer
        layer.enableTime("dateDatetime", "", "TRUE", None)
        # Set the start time for the layer
        layer.time.startTime = timeSettings["st"]
        # Set the end time for the layer
        layer.time.endTime = timeSettings["et"]
        # Set the time field for the layer
        layer.time.startTimeField = timeSettings["stf"]
        # Set the time step interval for the layer
        layer.time.timeStepInterval = timeSettings["tsi"]
        # Set the time step interval units for the layer
        layer.time.timeStepIntervalUnits = timeSettings["tsiu"]
        # Set the time zone for the layer
        layer.time.timeZone = timeSettings["tz"]

    # Re assign step interval and time units
    if layer.isTimeEnabled:
        layer.time.timeStepInterval = 1.0
        layer.time.timeStepIntervalUnits = "months"

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.2. Collisions Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the collisions map and set it as the active map

In [51]:
# Close all previous map views
aprx.closeViews()

# Open the roads map view
mapCollisions.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [52]:
for lyr in mapCollisions.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapCollisions.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [53]:
# Add the data layers to the map
mapCollisionsLyrBoundaries = mapCollisions.addDataFromPath(boundaries)
mapCollisionsLyrCities = mapCollisions.addDataFromPath(cities)
mapCollisionsLyrCensusBlocks = mapCollisions.addDataFromPath(censusBlocks)
mapCollisionsLyrRoads = mapCollisions.addDataFromPath(roads)
mapCollisionsLyrCollisions = mapCollisions.addDataFromPath(collisions)

# Set layer visibility on the map
mapCollisionsLyrBoundaries.visible = False
mapCollisionsLyrCities.visible = False
mapCollisionsLyrCensusBlocks.visible = False
mapCollisionsLyrRoads.visible = False
mapCollisionsLyrCollisions.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Enable Time Settings</h3>

Setup and enable time settings for the collisions map layers

In [54]:
setLayerTime(mapCollisionsLyrCollisions)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Collisions layer

In [55]:
# Apply the symbology for the Collisions data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCollisionsLyrCollisions,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Collisions.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:51:33 PM
Succeeded at Thursday, February 27, 2025 1:51:33 PM (Elapsed Time: 0.07 seconds)


- Roads layer

In [56]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCollisionsLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:51:34 PM
Succeeded at Thursday, February 27, 2025 1:51:34 PM (Elapsed Time: 0.03 seconds)


- Census Blocks layer

In [57]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCollisionsLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:51:34 PM
Succeeded at Thursday, February 27, 2025 1:51:34 PM (Elapsed Time: 0.04 seconds)


- Cities layer

In [58]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCollisionsLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:51:35 PM
Succeeded at Thursday, February 27, 2025 1:51:35 PM (Elapsed Time: 0.05 seconds)


- Boundaries layer

In [59]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCollisionsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:51:35 PM
Succeeded at Thursday, February 27, 2025 1:51:35 PM (Elapsed Time: 0.03 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [60]:
# Get CIM definitions for the collisions map layers
mapCollisionsCimBoundaries = mapCollisionsLyrBoundaries.getDefinition("V3")
mapCollisionsCimCities = mapCollisionsLyrCities.getDefinition("V3")
mapCollisionsCimCensusBlocks = mapCollisionsLyrCensusBlocks.getDefinition("V3")
mapCollisionsCimRoads = mapCollisionsLyrRoads.getDefinition("V3")
mapCollisionsCimCollisions = mapCollisionsLyrCollisions.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [61]:
# Set the layer headings
mapCollisionsCimBoundaries.renderer.heading = "Boundaries"
mapCollisionsCimCities.renderer.heading = "City Population Density"
mapCollisionsCimCensusBlocks.renderer.heading = "Population Density"
mapCollisionsCimRoads.renderer.heading = "Road Categories"
mapCollisionsCimCollisions.renderer.heading = "Severity Level"

# Update the map layer definitions
mapCollisionsLyrBoundaries.setDefinition(mapCollisionsCimBoundaries)
mapCollisionsLyrCities.setDefinition(mapCollisionsCimCities)
mapCollisionsLyrCensusBlocks.setDefinition(mapCollisionsCimCensusBlocks)
mapCollisionsLyrRoads.setDefinition(mapCollisionsCimRoads)
mapCollisionsLyrCollisions.setDefinition(mapCollisionsCimCollisions)

Update the CIM defintion for the collisions map

In [62]:
cimCollisions = mapCollisions.getDefinition("V3")  # Collisions map CIM

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [63]:
# Update the collisions map mapx file
exportCim("map", mapCollisions, "collisions")

Exporting collisions map to MAPX...
Start Time: Thursday, February 27, 2025 1:51:35 PM
Succeeded at Thursday, February 27, 2025 1:51:35 PM (Elapsed Time: 0.03 seconds)
Exporting collisions map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [64]:
for l in mapCollisions.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Collisions layer to LYRX...
Start Time: Thursday, February 27, 2025 1:51:36 PM
Succeeded at Thursday, February 27, 2025 1:51:36 PM (Elapsed Time: 0.16 seconds)
Exporting OCSWITRS Collisions layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:51:36 PM
Succeeded at Thursday, February 27, 2025 1:51:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:51:36 PM
Succeeded at Thursday, February 27, 2025 1:51:36 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:51:36 PM
Succeeded at Thursday, February 27, 2025 1:51:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:51:36 PM
Succee

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [65]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.3. Crashes Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the crashes map and set it as the active map

In [66]:
# Close all previous map views
aprx.closeViews()

# Open the crashes map view
mapCrashes.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [67]:
for lyr in mapCrashes.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapCrashes.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [68]:
# Add the data layers to the map
mapCrashesLyrBoundaries = mapCrashes.addDataFromPath(boundaries)
mapCrashesLyrCities = mapCrashes.addDataFromPath(cities)
mapCrashesLyrCensusBlocks = mapCrashes.addDataFromPath(censusBlocks)
mapCrashesLyrRoads = mapCrashes.addDataFromPath(roads)
mapCrashesLyrCrashes = mapCrashes.addDataFromPath(crashes)

# Set layer visibility on the map
mapCrashesLyrBoundaries.visible = False
mapCrashesLyrCities.visible = False
mapCrashesLyrCensusBlocks.visible = False
mapCrashesLyrRoads.visible = False
mapCrashesLyrCrashes.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Enable Time Settings</h3>

Setup and enable time settings for the crashes map layers

In [69]:
setLayerTime(mapCrashesLyrCrashes)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Crashes layer

In [70]:
# Apply the symbology for the Collisions data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCrashesLyrCrashes,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Crashes.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:53:36 PM
Succeeded at Thursday, February 27, 2025 1:53:36 PM (Elapsed Time: 0.10 seconds)


- Roads layer

In [71]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCrashesLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:53:36 PM
Succeeded at Thursday, February 27, 2025 1:53:36 PM (Elapsed Time: 0.03 seconds)


- Census Blocks layer

In [72]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCrashesLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:53:37 PM
Succeeded at Thursday, February 27, 2025 1:53:37 PM (Elapsed Time: 0.05 seconds)


- Cities layer

In [73]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCrashesLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:53:37 PM
Succeeded at Thursday, February 27, 2025 1:53:37 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [74]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapCrashesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:53:37 PM
Succeeded at Thursday, February 27, 2025 1:53:37 PM (Elapsed Time: 0.05 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [75]:
# Get CIM definitions for the crashes map layers
mapCrashesCimBoundaries = mapCrashesLyrBoundaries.getDefinition("V3")
mapCrashesCimCities = mapCrashesLyrCities.getDefinition("V3")
mapCrashesCimCensusBlocks = mapCrashesLyrCensusBlocks.getDefinition("V3")
mapCrashesCimRoads = mapCrashesLyrRoads.getDefinition("V3")
mapCrashesCimCrashes = mapCrashesLyrCrashes.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [76]:
# Set the layer headings
mapCrashesCimBoundaries.renderer.heading = "Boundaries"
mapCrashesCimCities.renderer.heading = "City Population Density"
mapCrashesCimCensusBlocks.renderer.heading = "Population Density"
mapCrashesCimRoads.renderer.heading = "Road Categories"
mapCrashesCimCrashes.renderer.heading = "Severity Level"

# Update the map layer definitions
mapCrashesLyrBoundaries.setDefinition(mapCrashesCimBoundaries)
mapCrashesLyrCities.setDefinition(mapCrashesCimCities)
mapCrashesLyrCensusBlocks.setDefinition(mapCrashesCimCensusBlocks)
mapCrashesLyrRoads.setDefinition(mapCrashesCimRoads)
mapCrashesLyrCrashes.setDefinition(mapCrashesCimCrashes)

Update the CIM defintion for the crashes map

In [77]:
cimCrashes = mapCrashes.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [78]:
# Update the crashes map mapx file
exportCim("map", mapCrashes, "crashes")

Exporting crashes map to MAPX...
Start Time: Thursday, February 27, 2025 1:53:37 PM
Succeeded at Thursday, February 27, 2025 1:53:37 PM (Elapsed Time: 0.05 seconds)
Exporting crashes map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [79]:
# Export the layers to JSON
for l in mapCrashes.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes layer to LYRX...
Start Time: Thursday, February 27, 2025 1:53:38 PM
Succeeded at Thursday, February 27, 2025 1:53:38 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Crashes layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:53:38 PM
Succeeded at Thursday, February 27, 2025 1:53:38 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:53:38 PM
Succeeded at Thursday, February 27, 2025 1:53:38 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:53:38 PM
Succeeded at Thursday, February 27, 2025 1:53:38 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:53:38 PM
Succeeded at

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [80]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.4. Parties Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the parties map and set it as the active map

In [81]:
# Close all previous map views
aprx.closeViews()

# Open the parties map view
mapParties.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [82]:
for lyr in mapParties.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapParties.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [83]:
# Add the data layers to the map
mapPartiesLyrBoundaries = mapParties.addDataFromPath(boundaries)
mapPartiesLyrCities = mapParties.addDataFromPath(cities)
mapPartiesLyrCensusBlocks = mapParties.addDataFromPath(censusBlocks)
mapPartiesLyrRoads = mapParties.addDataFromPath(roads)
mapPartiesLyrParties = mapParties.addDataFromPath(parties)

# Set layer visibility on the map
mapPartiesLyrBoundaries.visible = False
mapPartiesLyrCities.visible = False
mapPartiesLyrCensusBlocks.visible = False
mapPartiesLyrRoads.visible = False
mapPartiesLyrParties.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Enable Time Settings</h3>

Setup and enable time settings for the parties map layers

In [84]:
setLayerTime(mapPartiesLyrParties)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Parties layer

In [85]:
# Apply the symbology for the Parties data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPartiesLyrParties,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Parties.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:54:34 PM
Succeeded at Thursday, February 27, 2025 1:54:34 PM (Elapsed Time: 0.08 seconds)


- Roads layer

In [86]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPartiesLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:54:34 PM
Succeeded at Thursday, February 27, 2025 1:54:34 PM (Elapsed Time: 0.05 seconds)


- Census Blocks layer

In [87]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPartiesLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:54:34 PM
Succeeded at Thursday, February 27, 2025 1:54:34 PM (Elapsed Time: 0.10 seconds)


- Cities layer

In [88]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPartiesLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:54:34 PM
Succeeded at Thursday, February 27, 2025 1:54:35 PM (Elapsed Time: 0.09 seconds)


- Boundaries layer

In [89]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPartiesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:54:35 PM
Succeeded at Thursday, February 27, 2025 1:54:35 PM (Elapsed Time: 0.03 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [90]:
# Get CIM definitions for the parties map layers
mapPartiesCimBoundaries = mapPartiesLyrBoundaries.getDefinition("V3")
mapPartiesCimCities = mapPartiesLyrCities.getDefinition("V3")
mapPartiesCimCensusBlocks = mapPartiesLyrCensusBlocks.getDefinition("V3")
mapPartiesCimRoads = mapPartiesLyrRoads.getDefinition("V3")
mapPartiesCimParties = mapPartiesLyrParties.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [91]:
# Set the layer headings
mapPartiesCimBoundaries.renderer.heading = "Boundaries"
mapPartiesCimCities.renderer.heading = "City Population Density"
mapPartiesCimCensusBlocks.renderer.heading = "Population Density"
mapPartiesCimRoads.renderer.heading = "Road Categories"
mapPartiesCimParties.renderer.heading = "Severity Level"

# Update the map layer definitions
mapPartiesLyrBoundaries.setDefinition(mapPartiesCimBoundaries)
mapPartiesLyrCities.setDefinition(mapPartiesCimCities)
mapPartiesLyrCensusBlocks.setDefinition(mapPartiesCimCensusBlocks)
mapPartiesLyrRoads.setDefinition(mapPartiesCimRoads)
mapPartiesLyrParties.setDefinition(mapPartiesCimParties)

Update the CIM defintion for the parties map

In [92]:
partiesCim = mapParties.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [93]:
# Update the parties map mapx file
exportCim("map", mapParties, "parties")

Exporting parties map to MAPX...
Start Time: Thursday, February 27, 2025 1:54:35 PM
Succeeded at Thursday, February 27, 2025 1:54:35 PM (Elapsed Time: 0.03 seconds)
Exporting parties map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [94]:
# Export the layers to JSON
for l in mapParties.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Parties layer to LYRX...
Start Time: Thursday, February 27, 2025 1:54:35 PM
Succeeded at Thursday, February 27, 2025 1:54:35 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Parties layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:54:35 PM
Succeeded at Thursday, February 27, 2025 1:54:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:54:36 PM
Succeeded at Thursday, February 27, 2025 1:54:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:54:36 PM
Succeeded at Thursday, February 27, 2025 1:54:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:54:36 PM
Succeeded at

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [95]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.5. Victims Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the victims map and set it as the active map

In [96]:
# Close all previous map views
aprx.closeViews()

# Open the victims map view
mapVictims.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [97]:
# Remove all the layers from the map
for lyr in mapVictims.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapVictims.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [98]:
# Add the data layers to the map
mapVictimsLyrBoundaries = mapVictims.addDataFromPath(boundaries)
mapVictimsLyrCities = mapVictims.addDataFromPath(cities)
mapVictimsLyrCensusBlocks = mapVictims.addDataFromPath(censusBlocks)
mapVictimsLyrRoads = mapVictims.addDataFromPath(roads)
mapVictimsLyrVictims = mapVictims.addDataFromPath(victims)

# Set layer visibility on the map
mapVictimsLyrBoundaries.visible = False
mapVictimsLyrCities.visible = False
mapVictimsLyrCensusBlocks.visible = False
mapVictimsLyrRoads.visible = False
mapVictimsLyrVictims.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Enable Time Settings</h3>

Setup and enable time settings for the victims map layers

In [99]:
setLayerTime(mapVictimsLyrVictims)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Victims layer

In [100]:
# Apply the symbology for the Victims data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapVictimsLyrVictims,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Victims.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:06 PM
Succeeded at Thursday, February 27, 2025 1:55:06 PM (Elapsed Time: 0.05 seconds)


- Roads layer

In [101]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapVictimsLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:06 PM
Succeeded at Thursday, February 27, 2025 1:55:06 PM (Elapsed Time: 0.05 seconds)


- Census Blocks layer

In [102]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapVictimsLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:07 PM
Succeeded at Thursday, February 27, 2025 1:55:07 PM (Elapsed Time: 0.12 seconds)


- Cities layer

In [103]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapVictimsLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:07 PM
Succeeded at Thursday, February 27, 2025 1:55:07 PM (Elapsed Time: 0.12 seconds)


- Boundaries layer

In [104]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapVictimsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:07 PM
Succeeded at Thursday, February 27, 2025 1:55:07 PM (Elapsed Time: 0.06 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [105]:
# Get CIM definitions for the victims map layers
mapVictimsCimBoundaries = mapVictimsLyrBoundaries.getDefinition("V3")
mapVictimsCimCities = mapVictimsLyrCities.getDefinition("V3")
mapVictimsCimCensusBlocks = mapVictimsLyrCensusBlocks.getDefinition("V3")
mapVictimsCimRoads = mapVictimsLyrRoads.getDefinition("V3")
mapVictimsCimVictims = mapVictimsLyrVictims.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [106]:
# Set the layer headings
mapVictimsCimBoundaries.renderer.heading = "Boundaries"
mapVictimsCimCities.renderer.heading = "City Population Density"
mapVictimsCimCensusBlocks.renderer.heading = "Population Density"
mapVictimsCimRoads.renderer.heading = "Road Categories"
mapVictimsCimVictims.renderer.heading = "Severity Level"

# Update the map layer definitions
mapVictimsLyrBoundaries.setDefinition(mapVictimsCimBoundaries)
mapVictimsLyrCities.setDefinition(mapVictimsCimCities)
mapVictimsLyrCensusBlocks.setDefinition(mapVictimsCimCensusBlocks)
mapVictimsLyrRoads.setDefinition(mapVictimsCimRoads)
mapVictimsLyrVictims.setDefinition(mapVictimsCimVictims)

Update the CIM defintion for the victims map

In [107]:
cimVictims = mapVictims.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [108]:
# Update the victims map mapx file
exportCim("map", mapVictims, "victims")

Exporting victims map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:07 PM
Succeeded at Thursday, February 27, 2025 1:55:07 PM (Elapsed Time: 0.06 seconds)
Exporting victims map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [109]:
# Export the layers to JSON
for l in mapVictims.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Victims layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:08 PM
Succeeded at Thursday, February 27, 2025 1:55:08 PM (Elapsed Time: 0.12 seconds)
Exporting OCSWITRS Victims layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:08 PM
Succeeded at Thursday, February 27, 2025 1:55:08 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:08 PM
Succeeded at Thursday, February 27, 2025 1:55:08 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:08 PM
Succeeded at Thursday, February 27, 2025 1:55:08 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:09 PM
Succeeded at

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [110]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.6. Injuries Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the injuries map and set it as the active map

In [111]:
# Close all previous map views
aprx.closeViews()

# Open the injuries map view
mapInjuries.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [112]:
# Remove all layers from the map
for lyr in mapInjuries.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapInjuries.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [113]:
# Add the data layers to the map
mapInjuriesLyrBoundaries = mapInjuries.addDataFromPath(boundaries)
mapInjuriesLyrCities = mapInjuries.addDataFromPath(cities)
mapInjuriesLyrCensusBlocks = mapInjuries.addDataFromPath(censusBlocks)
mapInjuriesLyrVictims = mapInjuries.addDataFromPath(victims)

# Set layer visibility on the map
mapInjuriesLyrBoundaries.visible = False
mapInjuriesLyrCities.visible = False
mapInjuriesLyrCensusBlocks.visible = False
mapInjuriesLyrVictims.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Victims layer

In [114]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapInjuriesLyrVictims,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Victims.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:10 PM
Succeeded at Thursday, February 27, 2025 1:55:10 PM (Elapsed Time: 0.09 seconds)


- Census Blocks layer

In [115]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapInjuriesLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:10 PM
Succeeded at Thursday, February 27, 2025 1:55:11 PM (Elapsed Time: 0.13 seconds)


- Cities layer

In [116]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapInjuriesLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:11 PM
Succeeded at Thursday, February 27, 2025 1:55:11 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [117]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapInjuriesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:11 PM
Succeeded at Thursday, February 27, 2025 1:55:11 PM (Elapsed Time: 0.07 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [118]:
# Get CIM definitions for the regression map layers
mapInjuriesCimBoundaries = mapInjuriesLyrBoundaries.getDefinition("V3")
mapInjuriesCimCities = mapInjuriesLyrCities.getDefinition("V3")
mapInjuriesCimCensusBlocks = mapInjuriesLyrCensusBlocks.getDefinition("V3")
mapInjuriesCimVictims = mapInjuriesLyrVictims.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [119]:
# Set the layer headings
mapInjuriesCimBoundaries.renderer.heading = "Boundaries"
mapInjuriesCimCities.renderer.heading = "City Population Density"
mapInjuriesCimCensusBlocks.renderer.heading = "Population Density"
mapInjuriesCimVictims.renderer.heading = "Severity Level"

# Update the map layer definitions
mapInjuriesLyrBoundaries.setDefinition(mapInjuriesCimBoundaries)
mapInjuriesLyrCities.setDefinition(mapInjuriesCimCities)
mapInjuriesLyrCensusBlocks.setDefinition(mapInjuriesCimCensusBlocks)
mapInjuriesLyrVictims.setDefinition(mapInjuriesCimVictims)

Update the CIM defintion for the regression map

In [120]:
cimInjuries = mapInjuries.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [121]:
# Update the victims map mapx file
exportCim("map", mapInjuries, "injuries")

Exporting injuries map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:11 PM
Succeeded at Thursday, February 27, 2025 1:55:11 PM (Elapsed Time: 0.07 seconds)
Exporting injuries map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [122]:
# Export the layers to JSON
for l in mapInjuries.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Victims layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:12 PM
Succeeded at Thursday, February 27, 2025 1:55:12 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Victims layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:12 PM
Succeeded at Thursday, February 27, 2025 1:55:12 PM (Elapsed Time: 0.28 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:12 PM
Succeeded at Thursday, February 27, 2025 1:55:13 PM (Elapsed Time: 0.10 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:13 PM
Succeeded at Thursday, February 27, 2025 1:55:13 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [123]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.7. Fatalities Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the fatalities map and set it as the active map

In [124]:
# Close all previous map views
aprx.closeViews()

# Open the fatalities map view
mapFatalities.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [125]:
# Remove all the layers from the map
for lyr in mapFatalities.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapFatalities.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [126]:
# Add the data layers to the map
mapFatalitiesLyrBoundaries = mapFatalities.addDataFromPath(boundaries)
mapFatalitiesLyrRoads = mapFatalities.addDataFromPath(roads)
mapFatalitiesLyrRoadsMajorBuffers = mapFatalities.addDataFromPath(roadsMajorBuffers)
mapFatalitiesLyrFatalities = mapFatalities.addDataFromPath(crashes)

# Set layer visibility on the map
mapFatalitiesLyrBoundaries.visible = False
mapFatalitiesLyrRoads.visible = False
mapFatalitiesLyrRoadsMajorBuffers.visible = False
mapFatalitiesLyrFatalities.visible = False

# Move layers
mapFatalities.moveLayer(
    reference_layer = mapFatalitiesLyrRoads,
    move_layer = mapFatalitiesLyrRoadsMajorBuffers,
    insert_position = "BEFORE"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Fatalities layer

In [127]:
# Apply the symbology for the Fatalities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFatalitiesLyrFatalities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Crashes Killed Victims.lyrx"),
    symbology_fields = [["VALUE_FIELD", "numberKilled", "numberKilled"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:15 PM
Succeeded at Thursday, February 27, 2025 1:55:15 PM (Elapsed Time: 0.11 seconds)


- Major Road Buffers Layer

In [128]:
# Apply the symbology for the Major Road Buffers data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFatalitiesLyrRoadsMajorBuffers,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads Buffers.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:15 PM
Succeeded at Thursday, February 27, 2025 1:55:15 PM (Elapsed Time: 0.05 seconds)


- Roads layer

In [129]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFatalitiesLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:15 PM
Succeeded at Thursday, February 27, 2025 1:55:16 PM (Elapsed Time: 0.05 seconds)


- Boundaries layer

In [130]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFatalitiesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:16 PM
Succeeded at Thursday, February 27, 2025 1:55:16 PM (Elapsed Time: 0.05 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [131]:
# Get CIM definitions for the fatalities map layers
mapFatalitiesCimBoundaries = mapFatalitiesLyrBoundaries.getDefinition("V3")
mapFatalitiesCimRoads = mapFatalitiesLyrRoads.getDefinition("V3")
mapFatalitiesCimRoadsMajorBuffers = mapFatalitiesLyrRoadsMajorBuffers.getDefinition("V3")
mapFatalitiesCimFatalities = mapFatalitiesLyrFatalities.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [132]:
# Set the layer headings
mapFatalitiesCimBoundaries.renderer.heading = "Boundaries"
mapFatalitiesCimRoads.renderer.heading = "Road Categories"
mapFatalitiesCimRoadsMajorBuffers.renderer.heading = "Major Road Buffers"
mapFatalitiesCimFatalities.renderer.heading = "Victims Killed"

# Update the map layer definitions
mapFatalitiesLyrBoundaries.setDefinition(mapFatalitiesCimBoundaries)
mapFatalitiesLyrRoads.setDefinition(mapFatalitiesCimRoads)
mapFatalitiesLyrRoadsMajorBuffers.setDefinition(mapFatalitiesCimRoadsMajorBuffers)
mapFatalitiesLyrFatalities.setDefinition(mapFatalitiesCimFatalities)

Update the CIM defintion for the fatalities map

In [133]:
cimFatalities = mapFatalities.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [134]:
# Update the fatalities map mapx file
exportCim("map", mapFatalities, "fatalities")

Exporting fatalities map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:16 PM
Succeeded at Thursday, February 27, 2025 1:55:16 PM (Elapsed Time: 0.05 seconds)
Exporting fatalities map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [135]:
# Export the layers to JSON
for l in mapFatalities.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:16 PM
Succeeded at Thursday, February 27, 2025 1:55:17 PM (Elapsed Time: 0.09 seconds)
Exporting OCSWITRS Crashes layer to JSON...

Exporting OCSWITRS Major Roads Buffers layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:17 PM
Succeeded at Thursday, February 27, 2025 1:55:17 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads Buffers layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:17 PM
Succeeded at Thursday, February 27, 2025 1:55:17 PM (Elapsed Time: 0.22 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:17 PM
Succeeded at Thursday, February 27, 2025 1:55:18 PM (Elapsed Time: 0.30 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [136]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.8. Hotspots (100m, 1km) Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the hotspots (100m, 1km) map and set it as the active map

In [137]:
# Close all previous map views
aprx.closeViews()

# Open the hotspots100m1km map view
mapFhs100m1km.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [138]:
# Remove all the layers from the map
for lyr in mapFhs100m1km.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapFhs100m1km.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [139]:
# Add the data layers to the map
mapFhs100m1kmLyrBoundaries = mapFhs100m1km.addDataFromPath(boundaries)
mapFhs100m1kmLyrCities = mapFhs100m1km.addDataFromPath(cities)
mapFhs100m1kmLyrCensusBlocks = mapFhs100m1km.addDataFromPath(censusBlocks)
mapFhs100m1kmLyrRoads = mapFhs100m1km.addDataFromPath(roads)
mapFhs100m1kmLyrFhs100m1km = mapFhs100m1km.addDataFromPath(crashesFindHotspots100m1km)

# Set layer visibility on the map
mapFhs100m1kmLyrBoundaries.visible = False
mapFhs100m1kmLyrCities.visible = False
mapFhs100m1kmLyrCensusBlocks.visible = False
mapFhs100m1kmLyrRoads.visible = False
mapFhs100m1kmLyrFhs100m1km.visible = False

# Move layers
mapFhs100m1km.moveLayer(
    reference_layer = mapFhs100m1kmLyrBoundaries,
    move_layer = mapFhs100m1kmLyrRoads,
    insert_position = "AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Roads layer

In [140]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m1kmLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:19 PM
Succeeded at Thursday, February 27, 2025 1:55:19 PM (Elapsed Time: 0.06 seconds)


- Census Blocks layer

In [141]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m1kmLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:19 PM
Succeeded at Thursday, February 27, 2025 1:55:19 PM (Elapsed Time: 0.09 seconds)


- Cities layer

In [142]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m1kmLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:20 PM
Succeeded at Thursday, February 27, 2025 1:55:20 PM (Elapsed Time: 0.08 seconds)


- Boundaries layer

In [143]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m1kmLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:20 PM
Succeeded at Thursday, February 27, 2025 1:55:20 PM (Elapsed Time: 0.03 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [144]:
# Get CIM definitions for the hotspots100m1km map layers
mapFhs100m1kmCimBoundaries = mapFhs100m1kmLyrBoundaries.getDefinition("V3")
mapFhs100m1kmCimCities = mapFhs100m1kmLyrCities.getDefinition("V3")
mapFhs100m1kmCimCensusBlocks = mapFhs100m1kmLyrCensusBlocks.getDefinition("V3")
mapFhs100m1kmCimRoads = mapFhs100m1kmLyrRoads.getDefinition("V3")
mapFhs100m1kmCimFhs100m1km = mapFhs100m1kmLyrFhs100m1km.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [145]:
# Set the layer headings
mapFhs100m1kmCimBoundaries.renderer.heading = "Boundaries"
mapFhs100m1kmCimCities.renderer.heading = "City Population Density"
mapFhs100m1kmCimCensusBlocks.renderer.heading = "Population Density"
mapFhs100m1kmCimRoads.renderer.heading = "Road Categories"
mapFhs100m1kmCimFhs100m1km.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapFhs100m1kmLyrBoundaries.setDefinition(mapFhs100m1kmCimBoundaries)
mapFhs100m1kmLyrCities.setDefinition(mapFhs100m1kmCimCities)
mapFhs100m1kmLyrCensusBlocks.setDefinition(mapFhs100m1kmCimCensusBlocks)
mapFhs100m1kmLyrRoads.setDefinition(mapFhs100m1kmCimRoads)
mapFhs100m1kmLyrFhs100m1km.setDefinition(mapFhs100m1kmCimFhs100m1km)

Update the CIM defintion for the hotspots (100m, 1km) map

In [146]:
cimFhs100m1km = mapFhs100m1km.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [147]:
# Update the victims map mapx file
exportCim("map", mapFhs100m1km, "hotspots100m1km")

Exporting hotspots100m1km map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:20 PM
Succeeded at Thursday, February 27, 2025 1:55:20 PM (Elapsed Time: 0.03 seconds)
Exporting hotspots100m1km map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [148]:
# Export the layers to JSON
for l in mapFhs100m1km.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Find Hot Spots 100m 1km layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:21 PM
Succeeded at Thursday, February 27, 2025 1:55:21 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Crashes Find Hot Spots 100m 1km layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:21 PM
Succeeded at Thursday, February 27, 2025 1:55:21 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:21 PM
Succeeded at Thursday, February 27, 2025 1:55:21 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:21 PM
Succeeded at Thursday, February 27, 2025 1:55:21 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Boundaries layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [149]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.9. Hotspots (150m, 2km) Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the hotspots (150m, 2km) map and set it as the active map

In [150]:
# Close all previous map views
aprx.closeViews()

# Open the hotspots150m2km map view
mapFhs150m2km.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [151]:
# Remove all the layers from the map
for lyr in mapFhs150m2km.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapFhs150m2km.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [152]:
# Add the data layers to the map
mapFhs150m2kmLyrBoundaries = mapFhs150m2km.addDataFromPath(boundaries)
mapFhs150m2kmLyrCities = mapFhs150m2km.addDataFromPath(cities)
mapFhs150m2kmLyrCensusBlocks = mapFhs150m2km.addDataFromPath(censusBlocks)
mapFhs150m2kmLyrRoads = mapFhs150m2km.addDataFromPath(roads)
mapFhs150m2kmLyrFhs150m2km = mapFhs150m2km.addDataFromPath(crashesFindHotspots150m2km)

# Set layer visibility on the map
mapFhs150m2kmLyrBoundaries.visible = False
mapFhs150m2kmLyrCities.visible = False
mapFhs150m2kmLyrCensusBlocks.visible = False
mapFhs150m2kmLyrRoads.visible = False
mapFhs150m2kmLyrFhs150m2km.visible = False

# Move layers
mapFhs150m2km.moveLayer(
    reference_layer = mapFhs150m2kmLyrBoundaries,
    move_layer = mapFhs150m2kmLyrRoads,
    insert_position = "AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Roads layer

In [153]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs150m2kmLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:24 PM
Succeeded at Thursday, February 27, 2025 1:55:24 PM (Elapsed Time: 0.04 seconds)


- Census Blocks layer

In [154]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs150m2kmLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:24 PM
Succeeded at Thursday, February 27, 2025 1:55:24 PM (Elapsed Time: 0.06 seconds)


- Cities layer

In [155]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs150m2kmLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:24 PM
Succeeded at Thursday, February 27, 2025 1:55:24 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [156]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs150m2kmLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:25 PM
Succeeded at Thursday, February 27, 2025 1:55:25 PM (Elapsed Time: 0.04 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [157]:
# Get CIM definitions for the hotspots150m2km map layers
mapFhs150m2kmCimBoundaries = mapFhs150m2kmLyrBoundaries.getDefinition("V3")
mapFhs150m2kmCimCities = mapFhs150m2kmLyrCities.getDefinition("V3")
mapFhs150m2kmCimCensusBlocks = mapFhs150m2kmLyrCensusBlocks.getDefinition("V3")
mapFhs150m2kmCimRoads = mapFhs150m2kmLyrRoads.getDefinition("V3")
mapFhs150m2kmCimFhs150m2km = mapFhs150m2kmLyrFhs150m2km.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [158]:
# Set the layer headings
mapFhs150m2kmCimBoundaries.renderer.heading = "Boundaries"
mapFhs150m2kmCimCities.renderer.heading = "City Population Density"
mapFhs150m2kmCimCensusBlocks.renderer.heading = "Population Density"
mapFhs150m2kmCimRoads.renderer.heading = "Road Categories"
mapFhs150m2kmCimFhs150m2km.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapFhs150m2kmLyrBoundaries.setDefinition(mapFhs150m2kmCimBoundaries)
mapFhs150m2kmLyrCities.setDefinition(mapFhs150m2kmCimCities)
mapFhs150m2kmLyrCensusBlocks.setDefinition(mapFhs150m2kmCimCensusBlocks)
mapFhs150m2kmLyrRoads.setDefinition(mapFhs150m2kmCimRoads)
mapFhs150m2kmLyrFhs150m2km.setDefinition(mapFhs150m2kmCimFhs150m2km)

Update the CIM defintion for the hotspots (150m, 2km) map

In [159]:
cimFhs150m2km = mapFhs150m2km.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [160]:
# Update the victims map mapx file
exportCim("map", mapFhs150m2km, "hotspots150m2km")

Exporting hotspots150m2km map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:25 PM
Succeeded at Thursday, February 27, 2025 1:55:25 PM (Elapsed Time: 0.04 seconds)
Exporting hotspots150m2km map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [161]:
# Export the layers to JSON
for l in mapFhs150m2km.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Find Hot Spots 150m 2km layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:25 PM
Succeeded at Thursday, February 27, 2025 1:55:25 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Crashes Find Hot Spots 150m 2km layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:25 PM
Succeeded at Thursday, February 27, 2025 1:55:25 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:26 PM
Succeeded at Thursday, February 27, 2025 1:55:26 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:26 PM
Succeeded at Thursday, February 27, 2025 1:55:26 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Boundaries layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [162]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.10. Hotspots (100m, 5km) Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the hotspots (100m, 5km) map and set it as the active map

In [163]:
# Close all previous map views
aprx.closeViews()

# Open the hotspots100m5km map view
mapFhs100m5km.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [164]:
# Remove all the layers from the map
for lyr in mapFhs100m5km.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapFhs100m5km.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [165]:
# Add the data layers to the map
mapFhs100m5kmLyrBoundaries = mapFhs100m5km.addDataFromPath(boundaries)
mapFhs100m5kmLyrCities = mapFhs100m5km.addDataFromPath(cities)
mapFhs100m5kmLyrCensusBlocks = mapFhs100m5km.addDataFromPath(censusBlocks)
mapFhs100m5kmLyrRoads = mapFhs100m5km.addDataFromPath(roads)
mapFhs100m5kmLyrFhs100m5km = mapFhs100m5km.addDataFromPath(crashesFindHotspots100m5km)

# Set layer visibility on the map
mapFhs100m5kmLyrBoundaries.visible = False
mapFhs100m5kmLyrCities.visible = False
mapFhs100m5kmLyrCensusBlocks.visible = False
mapFhs100m5kmLyrRoads.visible = False
mapFhs100m5kmLyrFhs100m5km.visible = False

# Move layers
mapFhs100m5km.moveLayer(
    reference_layer = mapFhs100m5kmLyrBoundaries,
    move_layer = mapFhs100m5kmLyrRoads,
    insert_position = "AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Roads layer

In [166]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m5kmLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:27 PM
Succeeded at Thursday, February 27, 2025 1:55:27 PM (Elapsed Time: 0.05 seconds)


- Census Blocks layer

In [167]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m5kmLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:27 PM
Succeeded at Thursday, February 27, 2025 1:55:28 PM (Elapsed Time: 0.08 seconds)


- Cities layer

In [168]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m5kmLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:28 PM
Succeeded at Thursday, February 27, 2025 1:55:28 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [169]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhs100m5kmLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:28 PM
Succeeded at Thursday, February 27, 2025 1:55:28 PM (Elapsed Time: 0.12 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [170]:
# Get CIM definitions for the hotspots100m5km map layers
mapFhs100m5kmCimBoundaries = mapFhs100m5kmLyrBoundaries.getDefinition("V3")
mapFhs100m5kmCimCities = mapFhs100m5kmLyrCities.getDefinition("V3")
mapFhs100m5kmCimCensusBlocks = mapFhs100m5kmLyrCensusBlocks.getDefinition("V3")
mapFhs100m5kmCimRoads = mapFhs100m5kmLyrRoads.getDefinition("V3")
mapFhs100m5kmCimFhs100m5km = mapFhs100m5kmLyrFhs100m5km.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [171]:
# Set the layer headings
mapFhs100m5kmCimBoundaries.renderer.heading = "Boundaries"
mapFhs100m5kmCimCities.renderer.heading = "City Population Density"
mapFhs100m5kmCimCensusBlocks.renderer.heading = "Population Density"
mapFhs100m5kmCimRoads.renderer.heading = "Road Categories"
mapFhs100m5kmCimFhs100m5km.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapFhs100m5kmLyrBoundaries.setDefinition(mapFhs100m5kmCimBoundaries)
mapFhs100m5kmLyrCities.setDefinition(mapFhs100m5kmCimCities)
mapFhs100m5kmLyrCensusBlocks.setDefinition(mapFhs100m5kmCimCensusBlocks)
mapFhs100m5kmLyrRoads.setDefinition(mapFhs100m5kmCimRoads)
mapFhs100m5kmLyrFhs100m5km.setDefinition(mapFhs100m5kmCimFhs100m5km)

Update the CIM defintion for the hotspots (100m, 5km) map

In [172]:
cimFhs100m5kmLyr = mapFhs100m5km.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [173]:
# Update the victims map mapx file
exportCim("map", mapFhs100m5km, "hotspots100m5km")

Exporting hotspots100m5km map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:28 PM
Succeeded at Thursday, February 27, 2025 1:55:28 PM (Elapsed Time: 0.12 seconds)
Exporting hotspots100m5km map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [174]:
# Export the layers to JSON
for l in mapFhs100m5km.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Find Hot Spots 100m 5km layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:29 PM
Succeeded at Thursday, February 27, 2025 1:55:29 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Crashes Find Hot Spots 100m 5km layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:29 PM
Succeeded at Thursday, February 27, 2025 1:55:29 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:29 PM
Succeeded at Thursday, February 27, 2025 1:55:29 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:29 PM
Succeeded at Thursday, February 27, 2025 1:55:29 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Boundaries layer to JSON...

Exporting OCSWITRS Roads layer to LYRX...
Start Time

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [175]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.11. Hotspots 500ft from Major Roads Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the hotspots 500ft from major roads map and set it as the active map

In [176]:
# Close all previous map views
aprx.closeViews()

# Open the hotspotsroads500ft map view
mapFhsRoads500ft.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [177]:
# Remove all the layers from the map
for lyr in mapFhsRoads500ft.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapFhsRoads500ft.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [178]:
# Add the data layers to the map
mapFhsRoads500ftLyrBoundaries = mapFhsRoads500ft.addDataFromPath(boundaries)
mapFhsRoads500ftLyrCities = mapFhsRoads500ft.addDataFromPath(cities)
mapFhsRoads500ftLyrCensusBlocks = mapFhsRoads500ft.addDataFromPath(censusBlocks)
mapFhsRoads500ftLyrRoads = mapFhsRoads500ft.addDataFromPath(roads)
mapFhsRoads500ftLyrFhsRoads500ft = mapFhsRoads500ft.addDataFromPath(crashesFindHotspots500ftMajorRoads500ft1mi)

# Set layer visibility on the map
mapFhsRoads500ftLyrBoundaries.visible = False
mapFhsRoads500ftLyrCities.visible = False
mapFhsRoads500ftLyrCensusBlocks.visible = False
mapFhsRoads500ftLyrRoads.visible = False
mapFhsRoads500ftLyrFhsRoads500ft.visible = False

# Move layers
mapFhsRoads500ft.moveLayer(
    reference_layer = mapFhsRoads500ftLyrBoundaries,
    move_layer = mapFhsRoads500ftLyrRoads,
    insert_position = "AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Roads layer

In [179]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhsRoads500ftLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:31 PM
Succeeded at Thursday, February 27, 2025 1:55:31 PM (Elapsed Time: 0.05 seconds)


- Census Blocks layer

In [180]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhsRoads500ftLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:31 PM
Succeeded at Thursday, February 27, 2025 1:55:31 PM (Elapsed Time: 0.04 seconds)


- Cities layer

In [181]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhsRoads500ftLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:31 PM
Succeeded at Thursday, February 27, 2025 1:55:31 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [182]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapFhsRoads500ftLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:31 PM
Succeeded at Thursday, February 27, 2025 1:55:31 PM (Elapsed Time: 0.03 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [183]:
# Get CIM definitions for the hotspotsroads500ft map layers
mapFhsRoads500ftCimBoundaries = mapFhsRoads500ftLyrBoundaries.getDefinition("V3")
mapFhsRoads500ftCimCities = mapFhsRoads500ftLyrCities.getDefinition("V3")
mapFhsRoads500ftCimCensusBlocks = mapFhsRoads500ftLyrCensusBlocks.getDefinition("V3")
mapFhsRoads500ftCimRoads = mapFhsRoads500ftLyrRoads.getDefinition("V3")
mapFhsRoads500ftCimFhsRoads500ft = mapFhsRoads500ftLyrFhsRoads500ft.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [184]:
# Set the layer headings
mapFhsRoads500ftCimBoundaries.renderer.heading = "Boundaries"
mapFhsRoads500ftCimCities.renderer.heading = "City Population Density"
mapFhsRoads500ftCimCensusBlocks.renderer.heading = "Population Density"
mapFhsRoads500ftCimRoads.renderer.heading = "Road Categories"
mapFhsRoads500ftCimFhsRoads500ft.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapFhsRoads500ftLyrBoundaries.setDefinition(mapFhsRoads500ftCimBoundaries)
mapFhsRoads500ftLyrCities.setDefinition(mapFhsRoads500ftCimCities)
mapFhsRoads500ftLyrCensusBlocks.setDefinition(mapFhsRoads500ftCimCensusBlocks)
mapFhsRoads500ftLyrRoads.setDefinition(mapFhsRoads500ftCimRoads)
mapFhsRoads500ftLyrFhsRoads500ft.setDefinition(mapFhsRoads500ftCimFhsRoads500ft)

Update the CIM defintion for the hotspots 500ft from major roads map

In [185]:
cimFhsRoads500ft = mapFhsRoads500ft.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [186]:
# Update the victims map mapx file
exportCim("map", mapFhsRoads500ft, "hotspotsroads500ft")

Exporting hotspotsroads500ft map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:31 PM
Succeeded at Thursday, February 27, 2025 1:55:31 PM (Elapsed Time: 0.03 seconds)
Exporting hotspotsroads500ft map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [187]:
# Export the layers to JSON
for l in mapFhsRoads500ft.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:32 PM
Succeeded at Thursday, February 27, 2025 1:55:32 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:32 PM
Succeeded at Thursday, February 27, 2025 1:55:32 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:32 PM
Succeeded at Thursday, February 27, 2025 1:55:32 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:32 PM
Succeeded at Thursday, February 27, 2025 1:55:32 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Boundaries layer to JSON...

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [188]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.12. Optimized Hotspots 500ft from Major Roads Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Open the optimized hotspots 500ft from major roads map and set it as the active map

In [189]:
# Close all previous map views
aprx.closeViews()

# Open the optimized hot spots map view
mapOhsRoads500ft.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [190]:
# Remove all the layers from the map
for lyr in mapOhsRoads500ft.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapOhsRoads500ft.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [191]:
# Add the data layers to the map
mapOhsRoads500ftLyrBoundaries = mapOhsRoads500ft.addDataFromPath(boundaries)
mapOhsRoads500ftLyrCities = mapOhsRoads500ft.addDataFromPath(cities)
mapOhsRoads500ftLyrCensusBlocks = mapOhsRoads500ft.addDataFromPath(censusBlocks)
mapOhsRoads500ftLyrRoads = mapOhsRoads500ft.addDataFromPath(roads)
mapOhsRoads500ftLyrOhsRoads500ft = mapOhsRoads500ft.addDataFromPath(crashesFindHotspots500ftMajorRoads500ft1mi)

# Set layer visibility on the map
mapOhsRoads500ftLyrBoundaries.visible = False
mapOhsRoads500ftLyrCities.visible = False
mapOhsRoads500ftLyrCensusBlocks.visible = False
mapOhsRoads500ftLyrRoads.visible = False
mapOhsRoads500ftLyrOhsRoads500ft.visible = False

# Move layers
mapOhsRoads500ft.moveLayer(
    reference_layer = mapOhsRoads500ftLyrBoundaries,
    move_layer = mapOhsRoads500ftLyrRoads,
    insert_position = "AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Boundaries layer

In [192]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapOhsRoads500ftLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{16570DD4-87CC-49BC-BA38-DDEA7AD90ABB}'>

- Cities layer

In [193]:
# Apply the symbology for the cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapOhsRoads500ftLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{826FFC89-F5FC-469E-8900-BBBF79EB8439}'>

- Census Blocks layer

In [194]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapOhsRoads500ftLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{F9551C47-788D-4FA8-979F-DD0C470E2BDC}'>

- Roads layer

In [195]:
# Apply the symbology for the roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapOhsRoads500ftLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{C8F4AAC8-9050-41C1-9B7C-E755708819F4}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [196]:
# Get CIM definitions for the optimized hot spots map layers
mapOhsRoads500ftCimBoundaries = mapOhsRoads500ftLyrBoundaries.getDefinition("V3")
mapOhsRoads500ftCimCities = mapOhsRoads500ftLyrCities.getDefinition("V3")
mapOhsRoads500ftCimCensusBlocks = mapOhsRoads500ftLyrCensusBlocks.getDefinition("V3")
mapOhsRoads500ftCimRoads = mapOhsRoads500ftLyrRoads.getDefinition("V3")
mapOhsRoads500ftCimOhsRoads500ft = mapOhsRoads500ftLyrOhsRoads500ft.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [197]:
# Set the layer headings
mapOhsRoads500ftCimBoundaries.renderer.heading = "Boundaries"
mapOhsRoads500ftCimCities.renderer.heading = "City Population Density"
mapOhsRoads500ftCimCensusBlocks.renderer.heading = "Population Density"
mapOhsRoads500ftCimRoads.renderer.heading = "Road Categories"
mapOhsRoads500ftCimOhsRoads500ft.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapOhsRoads500ftLyrBoundaries.setDefinition(mapOhsRoads500ftCimBoundaries)
mapOhsRoads500ftLyrCities.setDefinition(mapOhsRoads500ftCimCities)
mapOhsRoads500ftLyrCensusBlocks.setDefinition(mapOhsRoads500ftCimCensusBlocks)
mapOhsRoads500ftLyrRoads.setDefinition(mapOhsRoads500ftCimRoads)
mapOhsRoads500ftLyrOhsRoads500ft.setDefinition(mapOhsRoads500ftCimOhsRoads500ft)

Update the CIM defintion for the major roads map

In [198]:
cimOhsRoads500ft = mapOhsRoads500ft.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [199]:
# Update the optimized hot spots map mapx file
exportCim("map", mapOhsRoads500ft, "optimizedHotspotsRoads500ft")

Exporting optimizedHotspotsRoads500ft map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:35 PM
Succeeded at Thursday, February 27, 2025 1:55:35 PM (Elapsed Time: 0.06 seconds)
Exporting optimizedHotspotsRoads500ft map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [200]:
# Export the layers to JSON
for l in mapOhsRoads500ft.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:35 PM
Succeeded at Thursday, February 27, 2025 1:55:35 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Crashes Find Hot Spots 500 Feet from Major Roads 500ft 1mi layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:36 PM
Succeeded at Thursday, February 27, 2025 1:55:36 PM (Elapsed Time: 0.16 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:36 PM
Succeeded at Thursday, February 27, 2025 1:55:36 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:36 PM
Succeeded at Thursday, February 27, 2025 1:55:36 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [201]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.13. Major Road Crashes Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the regression map and set it as the active map

In [202]:
# Close all previous map views
aprx.closeViews()

# Open the road crashes map view
mapRoadCrashes.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [203]:
# Remove all the layers from the map
for lyr in mapRoadCrashes.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapRoadCrashes.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [204]:
# Add the data layers to the map
mapRoadCrashesLyrBoundaries = mapRoadCrashes.addDataFromPath(boundaries)
mapRoadCrashesLyrCensusBlocks = mapRoadCrashes.addDataFromPath(censusBlocks)
mapRoadCrashesLyrRoadsMajor = mapRoadCrashes.addDataFromPath(roadsMajor)
mapRoadCrashesLyrCrashes500ftRoads = mapRoadCrashes.addDataFromPath(crashes500ftFromMajorRoads)

# Set layer visibility on the map
mapRoadCrashesLyrBoundaries.visible = False
mapRoadCrashesLyrCensusBlocks.visible = False
mapRoadCrashesLyrRoadsMajor.visible = False
mapRoadCrashesLyrCrashes500ftRoads.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Crashes (500ft from Major Roads) layer

In [205]:
# Apply the symbology for the Crashes 500ft from Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadCrashesLyrCrashes500ftRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Crashes.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:37 PM
Succeeded at Thursday, February 27, 2025 1:55:38 PM (Elapsed Time: 0.08 seconds)


- Major Roads layer

In [206]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadCrashesLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:38 PM
Succeeded at Thursday, February 27, 2025 1:55:38 PM (Elapsed Time: 0.03 seconds)


- Census Blocks layer

In [207]:
# Apply the symbology for the census blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadCrashesLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:38 PM
Succeeded at Thursday, February 27, 2025 1:55:38 PM (Elapsed Time: 0.13 seconds)


- Boundaries layer

In [208]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadCrashesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:38 PM
Succeeded at Thursday, February 27, 2025 1:55:38 PM (Elapsed Time: 0.05 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [209]:
# Get CIM definitions for the regression map layers
mapRoadCrashesCimBoundaries = mapRoadCrashesLyrBoundaries.getDefinition("V3")
mapRoadCrashesCimCensusBlocks = mapRoadCrashesLyrCensusBlocks.getDefinition("V3")
mapRoadCrashesCimRoadsMajor = mapRoadCrashesLyrRoadsMajor.getDefinition("V3")
mapRoadCrashesCimCrashes500ftRoads = mapRoadCrashesLyrCrashes500ftRoads.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [210]:
# Set the layer headings
mapRoadCrashesCimBoundaries.renderer.heading = "Boundaries"
mapRoadCrashesCimCensusBlocks.renderer.heading = "Population Density"
mapRoadCrashesCimRoadsMajor.renderer.heading = "Major Roads"
mapRoadCrashesCimRoadsMajor.renderer.heading = "Severity Level"

# Update the map layer definitions
mapRoadCrashesLyrBoundaries.setDefinition(mapRoadCrashesCimBoundaries)
mapRoadCrashesLyrCensusBlocks.setDefinition(mapRoadCrashesCimCensusBlocks)
mapRoadCrashesLyrRoadsMajor.setDefinition(mapRoadCrashesCimRoadsMajor)
mapRoadCrashesLyrCrashes500ftRoads.setDefinition(mapRoadCrashesCimCrashes500ftRoads)

Update the CIM defintion for the regression map

In [211]:
cimRoadCrashes = mapRoadCrashes.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [212]:
# Update the victims map mapx file
exportCim("map", mapRoadCrashes, "roadCrashes")

Exporting roadCrashes map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:38 PM
Succeeded at Thursday, February 27, 2025 1:55:38 PM (Elapsed Time: 0.05 seconds)
Exporting roadCrashes map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [213]:
# Export the layers to JSON
for l in mapRoadCrashes.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes 500 Feet from Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:39 PM
Succeeded at Thursday, February 27, 2025 1:55:39 PM (Elapsed Time: 0.09 seconds)
Exporting OCSWITRS Crashes 500 Feet from Major Roads layer to JSON...

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:39 PM
Succeeded at Thursday, February 27, 2025 1:55:39 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:39 PM
Succeeded at Thursday, February 27, 2025 1:55:39 PM (Elapsed Time: 0.09 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:39 PM
Succeeded at Thursday, February 27, 2025 1:55:40 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [214]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.14. Major Road Hotspots Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the road hotspots map and set it as the active map

In [215]:
# Close all previous map views
aprx.closeViews()

# Open the road hotspots map view
mapRoadHotspots.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [216]:
# Remove all the layers from the map
for lyr in mapRoadHotspots.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapRoadHotspots.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [217]:
# Add the data layers to the map
mapRoadHotspotsLyrBoundaries = mapRoadHotspots.addDataFromPath(boundaries)
mapRoadHotspotsLyrCensusBlocks = mapRoadHotspots.addDataFromPath(censusBlocks)
mapRoadHotspotsLyrRoadsMajor = mapRoadHotspots.addDataFromPath(roadsMajor)
mapRoadHotspotsLyrCrashesHotspots = mapRoadHotspots.addDataFromPath(crashesHotspots500ftFromMajorRoads)

# Set layer visibility on the map
mapRoadHotspotsLyrBoundaries.visible = False
mapRoadHotspotsLyrCensusBlocks.visible = False
mapRoadHotspotsLyrRoadsMajor.visible = False
mapRoadHotspotsLyrCrashesHotspots.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Major Roads layer

In [218]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadHotspotsLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:41 PM
Succeeded at Thursday, February 27, 2025 1:55:41 PM (Elapsed Time: 0.09 seconds)


- Census Blocks layer

In [219]:
# Apply the symbology for the census blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadHotspotsLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:41 PM
Succeeded at Thursday, February 27, 2025 1:55:41 PM (Elapsed Time: 0.04 seconds)


- Boundaries layer

In [220]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadHotspotsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:41 PM
Succeeded at Thursday, February 27, 2025 1:55:41 PM (Elapsed Time: 0.04 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [221]:
# Get CIM definitions for the regression map layers
mapRoadHotspotsCimBoundaries = mapRoadHotspotsLyrBoundaries.getDefinition("V3")
mapRoadHotspotsCimCensusBlocks = mapRoadHotspotsLyrCensusBlocks.getDefinition("V3")
mapRoadHotspotsCimRoadsMajor = mapRoadHotspotsLyrRoadsMajor.getDefinition("V3")
mapRoadHotspotsCimCrashesHotspots = mapRoadHotspotsLyrCrashesHotspots.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [222]:
# Set the layer headings
mapRoadHotspotsCimBoundaries.renderer.heading = "Boundaries"
mapRoadHotspotsCimCensusBlocks.renderer.heading = "Population Density"
mapRoadHotspotsCimRoadsMajor.renderer.heading = "Major Roads"
mapRoadHotspotsCimCrashesHotspots.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapRoadHotspotsLyrBoundaries.setDefinition(mapRoadHotspotsCimBoundaries)
mapRoadHotspotsLyrCensusBlocks.setDefinition(mapRoadHotspotsCimCensusBlocks)
mapRoadHotspotsLyrRoadsMajor.setDefinition(mapRoadHotspotsCimRoadsMajor)
mapRoadHotspotsLyrCrashesHotspots.setDefinition(mapRoadHotspotsCimCrashesHotspots)

Update the CIM defintion for the regression map

In [223]:
cimRoadHotspots = mapRoadHotspots.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [224]:
# Update the victims map mapx file
exportCim("map", mapRoadHotspots, "regression")

Exporting regression map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:41 PM
Succeeded at Thursday, February 27, 2025 1:55:41 PM (Elapsed Time: 0.04 seconds)
Exporting regression map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [225]:
# Export the layers to JSON
for l in mapRoadHotspots.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Hot Spots 500 Feet from Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:42 PM
Succeeded at Thursday, February 27, 2025 1:55:42 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Crashes Hot Spots 500 Feet from Major Roads layer to JSON...

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:42 PM
Succeeded at Thursday, February 27, 2025 1:55:42 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:42 PM
Succeeded at Thursday, February 27, 2025 1:55:42 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:42 PM
Succeeded at Thursday, February 27, 2025 1:55:42 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [226]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.15. Major Road Buffers Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the regression map and set it as the active map

In [227]:
# Close all previous map views
aprx.closeViews()

# Open the regression map view
mapRoadBuffers.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [228]:
# Remove all the layers from the map
for lyr in mapRoadBuffers.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapRoadBuffers.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [229]:
# Add the data layers to the map
mapRoadBuffersLyrBoundaries = mapRoadBuffers.addDataFromPath(boundaries)
mapRoadBuffersLyrCensusBlocks = mapRoadBuffers.addDataFromPath(censusBlocks)
mapRoadBuffersLyrRoadsMajor = mapRoadBuffers.addDataFromPath(roadsMajor)
mapRoadBuffersLyrRoadBuffers = mapRoadBuffers.addDataFromPath(roadsMajorBuffersSum)

# Set layer visibility on the map
mapRoadBuffersLyrBoundaries.visible = False
mapRoadBuffersLyrCensusBlocks.visible = False
mapRoadBuffersLyrRoadsMajor.visible = False
mapRoadBuffersLyrRoadBuffers.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Road Buffer Fatalities layer

In [230]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadBuffersLyrRoadBuffers,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads Buffers Summary.lyrx"),
    symbology_fields = [["VALUE_FIELD", "sum_numberKilled", "sum_numberKilled"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:43 PM
Succeeded at Thursday, February 27, 2025 1:55:44 PM (Elapsed Time: 0.05 seconds)


* Major Roads layer

In [231]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadBuffersLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{DF189352-0B21-4346-BA37-2E908C209CAC}'>

- Census Blocks layer

In [232]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadBuffersLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:44 PM
Succeeded at Thursday, February 27, 2025 1:55:44 PM (Elapsed Time: 0.07 seconds)


- Boundaries layer

In [233]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadBuffersLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:45 PM
Succeeded at Thursday, February 27, 2025 1:55:45 PM (Elapsed Time: 0.04 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [234]:
# Get CIM definitions for the regression map layers
mapRoadBuffersCimBoundaries = mapRoadBuffersLyrBoundaries.getDefinition("V3")
mapRoadBuffersCimCensusBlocks = mapRoadBuffersLyrCensusBlocks.getDefinition("V3")
mapRoadBuffersCimRoadsMajor = mapRoadBuffersLyrRoadsMajor.getDefinition("V3")
mapRoadBuffersCimRoadBuffers = mapRoadBuffersLyrRoadBuffers.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [235]:
# Set the layer headings
mapRoadBuffersCimBoundaries.renderer.heading = "Boundaries"
mapRoadBuffersCimCensusBlocks.renderer.heading = "Population Density"
mapRoadBuffersCimRoadsMajor.renderer.heading = "Major Roads"
mapRoadBuffersCimRoadBuffers.renderer.heading = "Fatalities (entire segment)"

# Update the map layer definitions
mapRoadBuffersLyrBoundaries.setDefinition(mapRoadBuffersCimBoundaries)
mapRoadBuffersLyrCensusBlocks.setDefinition(mapRoadBuffersCimCensusBlocks)
mapRoadBuffersLyrRoadsMajor.setDefinition(mapRoadBuffersCimRoadsMajor)
mapRoadBuffersLyrRoadBuffers.setDefinition(mapRoadBuffersCimRoadBuffers)

Update the CIM defintion for the regression map

In [236]:
cimRoadBuffers = mapRoadBuffers.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [237]:
# Update the victims map mapx file
exportCim("map", mapRoadBuffers, "roadBuffers")

Exporting roadBuffers map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:45 PM
Succeeded at Thursday, February 27, 2025 1:55:45 PM (Elapsed Time: 0.04 seconds)
Exporting roadBuffers map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [238]:
# Export the layers to JSON
for l in mapRoadBuffers.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:45 PM
Succeeded at Thursday, February 27, 2025 1:55:45 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Major Roads Buffers Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:45 PM
Succeeded at Thursday, February 27, 2025 1:55:45 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads Buffers Summary layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:46 PM
Succeeded at Thursday, February 27, 2025 1:55:46 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:46 PM
Succeeded at Thursday, February 27, 2025 1:55:46 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [239]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.16. Major Road Segments Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the major road segments map and set it as the active map

In [240]:
# Close all previous map views
aprx.closeViews()

# Open the major road segments map view
mapRoadSegments.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [241]:
# Remove all the layers from the map
for lyr in mapRoadSegments.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
mapRoadSegments.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [242]:
# Add the data layers to the map
mapRoadSegmentsLyrBoundaries = mapRoadSegments.addDataFromPath(boundaries)
mapRoadSegmentsLyrCensusBlocks = mapRoadSegments.addDataFromPath(censusBlocks)
mapRoadSegmentsLyrRoadsMajor = mapRoadSegments.addDataFromPath(roadsMajor)
mapRoadSegmentsLyrRoadsMajorSplit = mapRoadSegments.addDataFromPath(roadsMajorSplitBufferSum)

# Set layer visibility on the map
mapRoadSegmentsLyrBoundaries.visible = False
mapRoadSegmentsLyrCensusBlocks.visible = False
mapRoadSegmentsLyrRoadsMajor.visible = False
mapRoadSegmentsLyrRoadsMajorSplit.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Major Roads Split Buffer Summary layer

In [243]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadSegmentsLyrRoadsMajorSplit,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads Split Buffer Summary.lyrx"),
    symbology_fields=[["VALUE_FIELD", "sum_victimCount", "sum_victimCount"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:47 PM
Succeeded at Thursday, February 27, 2025 1:55:47 PM (Elapsed Time: 0.06 seconds)


- Major Roads layer

In [244]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadSegmentsLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{26989FAB-1C34-45D4-8940-CC37844D883E}'>

- Census Blocks layer

In [245]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadSegmentsLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{5BB4C0C8-3C53-4762-BF63-50B1009A2037}'>

- Boundaries layer

In [246]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadSegmentsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{0196DE0E-8D94-49A4-9713-028BE635901D}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [247]:
# Get CIM definitions for the regression map layers
mapRoadSegmentsCimBoundaries = mapRoadSegmentsLyrBoundaries.getDefinition("V3")
mapRoadSegmentsCimCensusBlocks = mapRoadSegmentsLyrCensusBlocks.getDefinition("V3")
mapRoadSegmentsCimRoadsMajor = mapRoadSegmentsLyrRoadsMajor.getDefinition("V3")
mapRoadSegmentsCimRoadsMajorSplit = mapRoadSegmentsLyrRoadsMajorSplit.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [248]:
# Set the layer headings
mapRoadSegmentsCimBoundaries.renderer.heading = "Boundaries"
mapRoadSegmentsCimCensusBlocks.renderer.heading = "Population Density"
mapRoadSegmentsCimRoadsMajor.renderer.heading = "Major Roads"
mapRoadSegmentsCimRoadsMajorSplit.renderer.heading = "Victim Count per 1,000ft segment"

# Update the map layer definitions
mapRoadSegmentsLyrBoundaries.setDefinition(mapRoadSegmentsCimBoundaries)
mapRoadSegmentsLyrCensusBlocks.setDefinition(mapRoadSegmentsCimCensusBlocks)
mapRoadSegmentsLyrRoadsMajor.setDefinition(mapRoadSegmentsCimRoadsMajor)
mapRoadSegmentsLyrRoadsMajorSplit.setDefinition(mapRoadSegmentsCimRoadsMajorSplit)

Update the CIM defintion for the regression map

In [249]:
cimRoadSegments = mapRoadSegments.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [250]:
# Update the victims map mapx file
exportCim("map", mapRoadSegments, "roadSegments")

Exporting roadSegments map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:48 PM
Succeeded at Thursday, February 27, 2025 1:55:48 PM (Elapsed Time: 0.10 seconds)
Exporting roadSegments map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [251]:
# Export the layers to JSON
for l in mapRoadSegments.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:49 PM
Succeeded at Thursday, February 27, 2025 1:55:49 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Major Roads Split Buffer Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:49 PM
Succeeded at Thursday, February 27, 2025 1:55:49 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads Split Buffer Summary layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:49 PM
Succeeded at Thursday, February 27, 2025 1:55:49 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:49 PM
Succeeded at Thursday, February 27, 2025 1:55:49 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [252]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.17. Roads Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the roads map and set it as the active map

In [253]:
# Close all previous map views
aprx.closeViews()

# Open the roads map view
mapRoads.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [254]:
# Remove all the layers from the map
for lyr in mapRoads.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapRoads.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [255]:
# Add the data layers to the map
mapRoadsLyrRoadsMajor = mapRoads.addDataFromPath(roadsMajor)
mapRoadsLyrRoadsMajorBuffers = mapRoads.addDataFromPath(roadsMajorBuffers)
mapRoadsLyrRoadsMajorBuffersSum = mapRoads.addDataFromPath(roadsMajorBuffersSum)
mapRoadsLyrRoadsMajorPointsAlongLines = mapRoads.addDataFromPath(roadsMajorPointsAlongLines)
mapRoadsLyrRoadsMajorSplit = mapRoads.addDataFromPath(roadsMajorSplit)
mapRoadsLyrRoadsMajorSplitBuffer = mapRoads.addDataFromPath(roadsMajorSplitBuffer)
mapRoadsLyrRoadsMajorSplitBufferSum = mapRoads.addDataFromPath(roadsMajorSplitBufferSum)

# Set layer visibility on the map
mapRoadsLyrRoadsMajor.visible = False
mapRoadsLyrRoadsMajorBuffers.visible = False
mapRoadsLyrRoadsMajorBuffersSum.visible = False
mapRoadsLyrRoadsMajorPointsAlongLines.visible = False
mapRoadsLyrRoadsMajorSplit.visible = False
mapRoadsLyrRoadsMajorSplitBuffer.visible = False
mapRoadsLyrRoadsMajorSplitBufferSum.visible = False

# move the layers
mapRoads.moveLayer(
    reference_layer = mapRoadsLyrRoadsMajorBuffers, 
    move_layer = mapRoadsLyrRoadsMajorBuffersSum, 
    insert_position="BEFORE"
)
mapRoads.moveLayer(
    reference_layer = mapRoadsLyrRoadsMajorBuffersSum,
    move_layer = mapRoadsLyrRoadsMajorPointsAlongLines, 
    insert_position="BEFORE"
)
mapRoads.moveLayer(
    reference_layer = mapRoadsLyrRoadsMajorBuffersSum, 
    move_layer = mapRoadsLyrRoadsMajorSplitBuffer, 
    insert_position="BEFORE"
)
mapRoads.moveLayer(
    reference_layer = mapRoadsLyrRoadsMajorSplitBuffer,
    move_layer = mapRoadsLyrRoadsMajorSplitBufferSum, 
    insert_position="BEFORE"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Major Roads layer

In [256]:
# Apply the symbology for the Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadsLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:51 PM
Succeeded at Thursday, February 27, 2025 1:55:51 PM (Elapsed Time: 0.08 seconds)


- Major Road Split layer

In [257]:
# Apply the symbology for the major roads layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRoadsLyrRoadsMajorSplit,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:51 PM
Succeeded at Thursday, February 27, 2025 1:55:51 PM (Elapsed Time: 0.04 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [258]:
# Get CIM definitions for the roads map layers
mapRoadsCimRoadsMajor = mapRoadsLyrRoadsMajor.getDefinition("V3")
mapRoadsCimRoadsMajorBuffers = mapRoadsLyrRoadsMajorBuffers.getDefinition("V3")
mapRoadsCimRoadsMajorBuffersSum = mapRoadsLyrRoadsMajorBuffersSum.getDefinition("V3")
mapRoadsCimRoadsMajorPointsAlongLines = mapRoadsLyrRoadsMajorPointsAlongLines.getDefinition("V3")
mapRoadsCimRoadsMajorSplit = mapRoadsLyrRoadsMajorSplit.getDefinition("V3")
mapRoadsCimRoadsMajorSplitBuffer = mapRoadsLyrRoadsMajorSplitBuffer.getDefinition("V3")
mapRoadsCimRoadsMajorSplitBufferSum = mapRoadsLyrRoadsMajorSplitBufferSum.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [259]:
# Set the layer headings
mapRoadsCimRoadsMajor.renderer.heading = "Road Category"
mapRoadsCimRoadsMajorBuffers.renderer.heading = "Major Road Buffers"
mapRoadsCimRoadsMajorBuffersSum.renderer.heading = "Major Road Buffers"
mapRoadsCimRoadsMajorPointsAlongLines.renderer.heading = "Major Road Points Along Lines"
mapRoadsCimRoadsMajorSplit.renderer.heading = "Road Category"
mapRoadsCimRoadsMajorSplitBuffer.renderer.heading = "Major Road Buffers"

# Update the map layer definitions
mapRoadsLyrRoadsMajor.setDefinition(mapRoadsCimRoadsMajor)
mapRoadsLyrRoadsMajorBuffers.setDefinition(mapRoadsCimRoadsMajorBuffers)
mapRoadsLyrRoadsMajorBuffersSum.setDefinition(mapRoadsCimRoadsMajorBuffersSum)
mapRoadsLyrRoadsMajorPointsAlongLines.setDefinition(mapRoadsCimRoadsMajorPointsAlongLines)
mapRoadsLyrRoadsMajorSplit.setDefinition(mapRoadsCimRoadsMajorSplit)
mapRoadsLyrRoadsMajorSplitBuffer.setDefinition(mapRoadsCimRoadsMajorSplitBuffer)
mapRoadsLyrRoadsMajorSplitBufferSum.setDefinition(mapRoadsCimRoadsMajorSplitBufferSum)

Update the CIM defintion for the roads map

In [260]:
cimRoads = mapRoads.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [261]:
# Update the victims map mapx file
exportCim("map", mapRoads, "roads")

Exporting roads map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:51 PM
Succeeded at Thursday, February 27, 2025 1:55:51 PM (Elapsed Time: 0.04 seconds)
Exporting roads map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [262]:
# Export the layers to JSON
for l in mapRoads.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads Split layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:52 PM
Succeeded at Thursday, February 27, 2025 1:55:52 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads Split layer to JSON...

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:52 PM
Succeeded at Thursday, February 27, 2025 1:55:52 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Major Roads Points Along Lines layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:52 PM
Succeeded at Thursday, February 27, 2025 1:55:52 PM (Elapsed Time: 0.04 seconds)
Exporting OCSWITRS Major Roads Points Along Lines layer to JSON...

Exporting OCSWITRS Major Roads Split Buffer Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:52 PM
Succeeded at Thursday, February 27, 2025 1:55:52 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Major Roads Split Buffer Summary layer to J

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [263]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.18. Hotspot Points Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the hotspot points map and set it as the active map

In [264]:
# Close all previous map views
aprx.closeViews()

# Open the hotspot points map view
mapPointFhs.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [265]:
# Remove all the layers from the map
for lyr in mapPointFhs.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapPointFhs.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [266]:
# Add the data layers to the map
mapPointFhsLyrBoundaries = mapPointFhs.addDataFromPath(boundaries)
mapPointFhsLyrRoadsMajor = mapPointFhs.addDataFromPath(roadsMajor)
mapPointFhsLyrFhs = mapPointFhs.addDataFromPath(crashesHotspots)

# Set layer visibility on the map
mapPointFhsLyrBoundaries.visible = False
mapPointFhsLyrRoadsMajor.visible = False
mapPointFhsLyrFhs.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Major Roads layer

In [267]:
# Apply the symbology for the Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPointFhsLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{7B6B4239-BC07-46F8-A1E7-01851384BEED}'>

- Boundaries layer

In [268]:
# Apply the symbology for the boundaires data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPointFhsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{12EC293E-AD17-4F15-ABC1-1E730272CCBA}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [269]:
# Get CIM definitions for the major roads map layers
mapPointFhsCimBoundaries = mapPointFhsLyrBoundaries.getDefinition("V3")
mapPointFhsCimRoadsMajor = mapPointFhsLyrRoadsMajor.getDefinition("V3")
mapPointFhsCimFhs = mapPointFhsLyrFhs.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [270]:
# Set the layer headings
mapPointFhsCimBoundaries.renderer.heading = "Boundaries"
mapPointFhsCimRoadsMajor.renderer.heading = "Major Roads"
mapPointFhsCimFhs.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapPointFhsLyrBoundaries.setDefinition(mapPointFhsCimBoundaries)
mapPointFhsLyrRoadsMajor.setDefinition(mapPointFhsCimRoadsMajor)
mapPointFhsLyrFhs.setDefinition(mapPointFhsCimFhs)

Update the CIM defintion for the major roads map

In [271]:
cimPointFhs = mapPointFhs.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [272]:
# Update the major roads map mapx file
exportCim("map", mapPointFhs, "pointFhs")

Exporting pointFhs map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:54 PM
Succeeded at Thursday, February 27, 2025 1:55:54 PM (Elapsed Time: 0.17 seconds)
Exporting pointFhs map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [273]:
# Export the layers to JSON
for l in mapPointFhs.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Hot Spots layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:55 PM
Succeeded at Thursday, February 27, 2025 1:55:55 PM (Elapsed Time: 0.12 seconds)
Exporting OCSWITRS Crashes Hot Spots layer to JSON...

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:55 PM
Succeeded at Thursday, February 27, 2025 1:55:55 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:56 PM
Succeeded at Thursday, February 27, 2025 1:55:56 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [274]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.19. Optimized Hotspot Points Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the optimized hotspot points map and set it as the active map

In [275]:
# Close all previous map views
aprx.closeViews()

# Open the optimized hot spot points map view
mapPointOhs.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [276]:
# Remove all the layers from the map
for lyr in mapPointOhs.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapPointOhs.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [277]:
# Add the data layers to the map
mapPointOhsLyrBoundaries = mapPointOhs.addDataFromPath(boundaries)
mapPointOhsLyrRoadsMajor = mapPointOhs.addDataFromPath(roadsMajor)
mapPointOhsLyrOhs = mapPointOhs.addDataFromPath(crashesOptimizedHotspots)

# Set layer visibility on the map
mapPointOhsLyrBoundaries.visible = False
mapPointOhsLyrRoadsMajor.visible = False
mapPointOhsLyrOhs.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Major Roads layer

In [278]:
# Apply the symbology for the Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPointOhsLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{59ECF3C5-5FDD-4BE4-911E-C9A00FE50554}'>

- Boundaries layer

In [279]:
# Apply the symbology for the boundaires data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPointOhsLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{6E7B5CA2-E5B6-47AD-B304-0593A2A8C4F9}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [280]:
# Get CIM definitions for the road buffers sum map layers
mapPointOhsCimBoundaries = mapPointOhsLyrBoundaries.getDefinition("V3")
mapPointOhsCimRoadsMajor = mapPointOhsLyrRoadsMajor.getDefinition("V3")
mapPointOhsCimOhs = mapPointOhsLyrOhs.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [281]:
# Set the layer headings
mapPointOhsCimBoundaries.renderer.heading = "Boundaries"
mapPointOhsCimRoadsMajor.renderer.heading = "Major Roads"
mapPointOhsCimOhs.renderer.heading = "Getis-Ord Gi*"

# Update the map layer definitions
mapPointOhsLyrBoundaries.setDefinition(mapPointOhsCimBoundaries)
mapPointOhsLyrRoadsMajor.setDefinition(mapPointOhsCimRoadsMajor)
mapPointOhsLyrOhs.setDefinition(mapPointOhsCimOhs)

Update the CIM defintion for the major roads map

In [282]:
cimPointOhs = mapPointOhs.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [283]:
# Update the road buffers map mapx file
exportCim("map", mapPointOhs, "pointOhs")

Exporting pointOhs map to MAPX...
Start Time: Thursday, February 27, 2025 1:55:57 PM
Succeeded at Thursday, February 27, 2025 1:55:57 PM (Elapsed Time: 0.04 seconds)
Exporting pointOhs map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [284]:
# Export the layers to JSON
for l in mapPointOhs.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Optimized Hot Spots layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:58 PM
Succeeded at Thursday, February 27, 2025 1:55:58 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Crashes Optimized Hot Spots layer to JSON...

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:58 PM
Succeeded at Thursday, February 27, 2025 1:55:58 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:55:58 PM
Succeeded at Thursday, February 27, 2025 1:55:58 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [285]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.20. Population Density Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the densities map and set it as the active map

In [286]:
# Close all previous map views
aprx.closeViews()

# Open the population densities map view
mapPopDens.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [287]:
# Remove all the layers from the map
for lyr in mapPopDens.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapPopDens.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [288]:
# Add the data layers to the map
mapPopDensLyrBoundaries = mapPopDens.addDataFromPath(boundaries)
mapPopDensLyrRoadsMajor = mapPopDens.addDataFromPath(roadsMajor)
mapPopDensLyrPopDens = mapPopDens.addDataFromPath(censusBlocksSum)

# Rename the layer
mapPopDensLyrPopDens.name = "OCSWITRS Population Density"

# Set layer visibility on the map
mapPopDensLyrBoundaries.visible = False
mapPopDensLyrRoadsMajor.visible = False
mapPopDensLyrPopDens.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Population Density layer

In [289]:
# Apply the symbology for the Housing Density data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPopDensLyrPopDens,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Population Density.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:55:59 PM
Succeeded at Thursday, February 27, 2025 1:55:59 PM (Elapsed Time: 0.08 seconds)


- Major Roads layer

In [290]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPopDensLyrRoadsMajor,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{8CC1E94D-3FB3-4B22-AFCA-4595A90DBF1F}'>

- Boundaries layer

In [291]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapPopDensLyrBoundaries,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:00 PM
Succeeded at Thursday, February 27, 2025 1:56:00 PM (Elapsed Time: 0.17 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [292]:
# Get CIM definitions for the densities map layers
mapPopDensCimBoundaries = mapPopDensLyrBoundaries.getDefinition("V3")
mapPopDensCimRoadsMajor = mapPopDensLyrRoadsMajor.getDefinition("V3")
mapPopDensCimPopDens = mapPopDensLyrPopDens.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [293]:
# Set the layer headings
mapPopDensCimBoundaries.renderer.heading = "Boundaries"
mapPopDensCimRoadsMajor.renderer.heading = "Major Roads"
mapPopDensCimPopDens.renderer.heading = "Population Density"

# Update the map layer definitions
mapPopDensLyrBoundaries.setDefinition(mapPopDensCimBoundaries)
mapPopDensLyrRoadsMajor.setDefinition(mapPopDensCimRoadsMajor)
mapPopDensLyrPopDens.setDefinition(mapPopDensCimPopDens)

Apply CIM operations to the layers in the densities map

In [294]:
cimPopDens = mapPopDens.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [295]:
# Update the densities map mapx file
exportCim("map", mapPopDens, "popDens")

Exporting popDens map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:00 PM
Succeeded at Thursday, February 27, 2025 1:56:00 PM (Elapsed Time: 0.17 seconds)
Exporting popDens map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [296]:
# Export the layers to JSON
for l in mapPopDens.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:00 PM
Succeeded at Thursday, February 27, 2025 1:56:00 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Population Density layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:01 PM
Succeeded at Thursday, February 27, 2025 1:56:01 PM (Elapsed Time: 0.09 seconds)
Exporting OCSWITRS Population Density layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:01 PM
Succeeded at Thursday, February 27, 2025 1:56:01 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [297]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.21. Housing Density Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the housing density map and set it as the active map

In [298]:
# Close all previous map views
aprx.closeViews()

# Open the housing density map view
mapHouDens.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [299]:
# Remove all the layers from the map
for lyr in mapHouDens.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapHouDens.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [300]:
# Add the data layers to the map
mapHouDensLyrBoundaries = mapHouDens.addDataFromPath(boundaries)
mapHouDensLyrRoadsMajor = mapHouDens.addDataFromPath(roadsMajor)
mapHouDensLyrHouDens = mapHouDens.addDataFromPath(censusBlocksSum)

# Rename the layer
mapHouDensLyrHouDens.name = "OCSWITRS Housing Density"

# Set layer visibility on the map
mapHouDensLyrBoundaries.visible = False
mapHouDensLyrRoadsMajor.visible = False
mapHouDensLyrHouDens.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Housing Density layer

In [301]:
# Apply the symbology for the Housing Density data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapHouDensLyrHouDens,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Housing Density.lyrx"),
    symbology_fields = [["VALUE_FIELD", "housingDensity", "housingDensity"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{BCD1E8A9-93B3-495B-B064-462454F4E0B4}'>

- Major Roads layer

In [302]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapHouDensLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{CB784D78-CAF6-4FD1-9E17-7021FBF6FFF7}'>

- Boundaries layer

In [303]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapHouDensLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{395F874E-F988-47C3-8AFB-178621D2BF72}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [304]:
# Get CIM definitions for the regression map layers
mapHouDensCimBoundaries = mapHouDensLyrBoundaries.getDefinition("V3")
mapHouDensCimRoadsMajor = mapHouDensLyrRoadsMajor.getDefinition("V3")
mapHouDensCimHouDens = mapHouDensLyrHouDens.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [305]:
# Set the layer headings
mapHouDensCimBoundaries.renderer.heading = "Boundaries"
mapHouDensCimRoadsMajor.renderer.heading = "Major Roads"
mapHouDensCimHouDens.renderer.heading = "Housing Density"

# Update the map layer definitions
mapHouDensLyrBoundaries.setDefinition(mapHouDensCimBoundaries)
mapHouDensLyrRoadsMajor.setDefinition(mapHouDensCimRoadsMajor)
mapHouDensLyrHouDens.setDefinition(mapHouDensCimHouDens)

Update the CIM defintion for the regression map

In [306]:
cimHouDens = mapHouDens.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [307]:
# Update the victims map mapx file
exportCim("map", mapHouDens, "houDens")

Exporting houDens map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:03 PM
Succeeded at Thursday, February 27, 2025 1:56:03 PM (Elapsed Time: 0.03 seconds)
Exporting houDens map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [308]:
# Export the layers to JSON
for l in mapHouDens.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:03 PM
Succeeded at Thursday, February 27, 2025 1:56:03 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Housing Density layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:03 PM
Succeeded at Thursday, February 27, 2025 1:56:03 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Housing Density layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:04 PM
Succeeded at Thursday, February 27, 2025 1:56:04 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [309]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.22. Victims by City Areas Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Open the city areas map and set it as the active map

In [310]:
# Close all previous map views
aprx.closeViews()

# Open the city areas map layers
mapAreaCities.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [311]:
# Remove all the layers from the map
for lyr in mapAreaCities.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapAreaCities.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [312]:
# Add the data layers to the map
mapAreaCitiesLyrBoundaries = mapAreaCities.addDataFromPath(boundaries)
mapAreaCitiesLyrRoadsMajor = mapAreaCities.addDataFromPath(roadsMajor)
mapAreaCitiesLyrCities = mapAreaCities.addDataFromPath(citiesSum)

# Set layer visibility on the map
mapAreaCitiesLyrBoundaries.visible = False
mapAreaCitiesLyrRoadsMajor.visible = False
mapAreaCitiesLyrCities.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Boundaries layer

In [313]:
# Apply the symbology for the boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaCitiesLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{3C67897D-DA41-4F3B-A5A5-04CE50EF1348}'>

- Major Roads layer

In [314]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaCitiesLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{50AAAEB2-103C-40D2-B692-D59222B59376}'>

- City Areas Summary layer

In [315]:
# Apply the symbology for the cities sum data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaCitiesLyrCities,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Cities Summary.lyrx"),
    symbology_fields = [["VALUE_FIELD", "sum_victimCount", "sum_victimCount"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{C4CF29C8-4442-4BF7-A35C-29D6E2EB0048}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [316]:
# Get CIM definitions for the cities map layers
mapAreaCitiesCimBoundaries = mapAreaCitiesLyrBoundaries.getDefinition("V3")
mapAreaCitiesCimRoadsMajor = mapAreaCitiesLyrRoadsMajor.getDefinition("V3")
mapAreaCitiesCimCities = mapAreaCitiesLyrCities.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [317]:
# Set the layer headings
mapAreaCitiesCimBoundaries.renderer.heading = "Boundaries"
mapAreaCitiesCimRoadsMajor.renderer.heading = "Major Roads"
mapAreaCitiesCimCities.renderer.heading = "Victim Count"

# Update the map layer definitions
mapAreaCitiesLyrBoundaries.setDefinition(mapAreaCitiesCimBoundaries)
mapAreaCitiesLyrRoadsMajor.setDefinition(mapAreaCitiesCimRoadsMajor)
mapAreaCitiesLyrCities.setDefinition(mapAreaCitiesCimCities)

Update the CIM defintion for the city area map

In [318]:
cimAreasCities = mapAreaCities.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [319]:
# Update the Cities map mapx file
exportCim("map", mapAreaCities, "areaCities")

Exporting areaCities map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:06 PM
Succeeded at Thursday, February 27, 2025 1:56:06 PM (Elapsed Time: 0.09 seconds)
Exporting areaCities map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [320]:
# Export the layers to JSON
for l in mapAreaCities.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:06 PM
Succeeded at Thursday, February 27, 2025 1:56:06 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Cities Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:06 PM
Succeeded at Thursday, February 27, 2025 1:56:06 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Cities Summary layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:07 PM
Succeeded at Thursday, February 27, 2025 1:56:07 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [321]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.22. Victims By Census Blocks Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Open the census blocks map and set it as the active map

In [322]:
# Close all previous map views
aprx.closeViews()

# Open the census blocks map view
mapAreaBlocks.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [323]:
# Remove all the layers from the map
for lyr in mapAreaBlocks.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapAreaBlocks.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [324]:
# Add the data layers to the map
mapAreaBlocksLyrBoundaries = mapAreaBlocks.addDataFromPath(boundaries)
mapAreaBlocksLyrRoadsMajor = mapAreaBlocks.addDataFromPath(roadsMajor)
mapAreaBlocksLyrCensusBlocks = mapAreaBlocks.addDataFromPath(censusBlocksSum)

# Set layer visibility on the map
mapAreaBlocksLyrBoundaries.visible = False
mapAreaBlocksLyrRoadsMajor.visible = False
mapAreaBlocksLyrCensusBlocks.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Boundaries layer

In [325]:
# Apply the symbology for the boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaBlocksLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{7FADC87A-FBB1-4A12-90D7-C68C0CEFE2BB}'>

- Major Roads layer

In [326]:
# Apply the symbology for the major roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaBlocksLyrRoadsMajor,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Major Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)

<Result 'GPLYR_{EF60882D-0ED4-411D-8266-590FF47442D7}'>

- Census Blocks Summary layer

In [327]:
# Apply the symbology for the census blocks sum data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapAreaBlocksLyrCensusBlocks,
    in_symbology_layer=os.path.join(
        layersTemplates, "OCSWITRS Census Blocks Summary.lyrx"),
    symbology_fields=[["VALUE_FIELD", "sum_victimCount", "sum_victimCount"]],
    update_symbology="DEFAULT"
)

<Result 'GPLYR_{4FF752A2-0366-4FF0-88BD-0BF3EF201C35}'>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [328]:
# Get CIM definitions for the blocks map layers
mapAreaBlocksCimBoundaries = mapAreaBlocksLyrBoundaries.getDefinition("V3")
mapAreaBlocksCimRoadsMajor = mapAreaBlocksLyrRoadsMajor.getDefinition("V3")
mapAreaBlocksCimCensusBlocks = mapAreaBlocksLyrCensusBlocks.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [329]:
# Set the layer headings
mapAreaBlocksCimBoundaries.renderer.heading = "Boundaries"
mapAreaBlocksCimRoadsMajor.renderer.heading = "Major Roads"
mapAreaBlocksCimCensusBlocks.renderer.heading = "Victim Count"

# Update the map layer definitions
mapAreaBlocksLyrBoundaries.setDefinition(mapAreaBlocksCimBoundaries)
mapAreaBlocksLyrRoadsMajor.setDefinition(mapAreaBlocksCimRoadsMajor)
mapAreaBlocksLyrCensusBlocks.setDefinition(mapAreaBlocksCimCensusBlocks)

Update the CIM defintion for the major roads map

In [330]:
cimAreaBlocks = mapAreaBlocks.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [331]:
# Update the Blocks map mapx file
exportCim("map", mapAreaBlocks, "areaBlocks")

Exporting areaBlocks map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:08 PM
Succeeded at Thursday, February 27, 2025 1:56:08 PM (Elapsed Time: 0.08 seconds)
Exporting areaBlocks map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [332]:
# Export the layers to JSON
for l in mapAreaBlocks.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:09 PM
Succeeded at Thursday, February 27, 2025 1:56:09 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Major Roads layer to JSON...

Exporting OCSWITRS Census Blocks Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:09 PM
Succeeded at Thursday, February 27, 2025 1:56:09 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Census Blocks Summary layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:09 PM
Succeeded at Thursday, February 27, 2025 1:56:09 PM (Elapsed Time: 0.10 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [333]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.23. Summaries Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the summaries map and set it as the active map

In [334]:
# Close all previous map views
aprx.closeViews()

# Open the summaries map view
mapSummaries.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [335]:
# Remove all the layers from the map
for lyr in mapSummaries.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapSummaries.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [336]:
# Add the data layers to the map
mapSummariesLyrCensusBlocksSum = mapSummaries.addDataFromPath(censusBlocksSum)
mapSummariesLyrCitiesSum = mapSummaries.addDataFromPath(citiesSum)
mapSummariesLyrCrashes500ftFromMajorRoads = mapSummaries.addDataFromPath(crashes500ftFromMajorRoads)

# Set layer visibility on the map
mapSummariesLyrCensusBlocksSum.visible = False
mapSummariesLyrCitiesSum.visible = False
mapSummariesLyrCrashes500ftFromMajorRoads.visible = False

# Move the layers
mapSummaries.moveLayer(
    reference_layer = mapSummariesLyrCensusBlocksSum,
    move_layer = mapSummariesLyrCitiesSum,
    insert_position="AFTER"
)

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Census Block Summary layer

In [337]:
# Apply the symbology for the Census 2020 Blocks summary layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapSummariesLyrCensusBlocksSum,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:11 PM
Succeeded at Thursday, February 27, 2025 1:56:11 PM (Elapsed Time: 0.08 seconds)


- Cities Summary layer

In [338]:
# Apply the symbology for the Cities summary layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapSummariesLyrCitiesSum,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:11 PM
Succeeded at Thursday, February 27, 2025 1:56:11 PM (Elapsed Time: 0.04 seconds)


- Crashes 500ft from Major Roads layer

In [339]:
# Apply the symbology for the Crashes 500ft from Major Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapSummariesLyrCrashes500ftFromMajorRoads,
    in_symbology_layer=os.path.join(layersTemplates, "OCSWITRS Collisions.lyrx"),
    symbology_fields = [["VALUE_FIELD", "collSeverity", "collSeverity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:11 PM
Succeeded at Thursday, February 27, 2025 1:56:11 PM (Elapsed Time: 0.24 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [340]:
# Get CIM definitions for the summaries map layers
mapSummariesCimCensusBlocksSum = mapSummariesLyrCensusBlocksSum.getDefinition("V3")
mapSummariesCimCitiesSum = mapSummariesLyrCitiesSum.getDefinition("V3")
mapSummariesCimCrashes500ftFromMajorRoads = mapSummariesLyrCrashes500ftFromMajorRoads.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [341]:
# Set the layer headings
mapSummariesCimCensusBlocksSum.renderer.heading = "Population Density"
mapSummariesCimCitiesSum.renderer.heading = "City Population Density"
mapSummariesCimCrashes500ftFromMajorRoads.renderer.heading = "Collision Severity"

# Update the map layer definitions
mapSummariesLyrCensusBlocksSum.setDefinition(mapSummariesCimCensusBlocksSum)
mapSummariesLyrCitiesSum.setDefinition(mapSummariesCimCitiesSum)
mapSummariesLyrCrashes500ftFromMajorRoads.setDefinition(mapSummariesCimCrashes500ftFromMajorRoads)

Update the CIM defintion for the summaries map

In [342]:
summariesCim = mapSummaries.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [343]:
# Update the victims map mapx file
exportCim("map", mapSummaries, "summaries")

Exporting summaries map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:11 PM
Succeeded at Thursday, February 27, 2025 1:56:11 PM (Elapsed Time: 0.24 seconds)
Exporting summaries map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [344]:
# Export the layers to JSON
for l in mapSummaries.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes 500 Feet from Major Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:12 PM
Succeeded at Thursday, February 27, 2025 1:56:12 PM (Elapsed Time: 0.10 seconds)
Exporting OCSWITRS Crashes 500 Feet from Major Roads layer to JSON...

Exporting OCSWITRS Census Blocks Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:12 PM
Succeeded at Thursday, February 27, 2025 1:56:12 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Census Blocks Summary layer to JSON...

Exporting OCSWITRS Cities Summary layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:12 PM
Succeeded at Thursday, February 27, 2025 1:56:12 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Cities Summary layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [345]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.24. Analysis Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the analysis map and set it as the active map

In [346]:
# Close all previous map views
aprx.closeViews()

# Open the analysis map view
mapAnalysis.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [347]:
# Remove all the layers from the map
for lyr in mapAnalysis.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapAnalysis.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [348]:
# Add the data layers to the map
mapAnalysisLyrCrashesHotspots = mapAnalysis.addDataFromPath(crashesHotspots)
mapAnalysisLyrCrashesOptimizedHotspots = mapAnalysis.addDataFromPath(crashesOptimizedHotspots)

# Set layer visibility on the map
mapAnalysisLyrCrashesHotspots.visible = False
mapAnalysisLyrCrashesOptimizedHotspots.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Update the CIM defintion for the analysis map

In [349]:
analysisCim = mapAnalysis.getDefinition("V3")

Generate CIM JSON configuration for layers

In [350]:
# Get CIM definitions for the analysis map layers
mapAnalysisCimCrashesHotspots = mapAnalysisLyrCrashesHotspots.getDefinition("V3")
mapAnalysisCimCrashesOptimizedHotspots = mapAnalysisLyrCrashesOptimizedHotspots.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [351]:
# Update the victims map mapx file
exportCim("map", mapAnalysis, "analysis")

Exporting analysis map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:12 PM
Succeeded at Thursday, February 27, 2025 1:56:12 PM (Elapsed Time: 0.07 seconds)
Exporting analysis map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [352]:
# Export the layers to JSON
for l in mapAnalysis.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Crashes Optimized Hot Spots layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:14 PM
Succeeded at Thursday, February 27, 2025 1:56:14 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Crashes Optimized Hot Spots layer to JSON...

Exporting OCSWITRS Crashes Hot Spots layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:14 PM
Succeeded at Thursday, February 27, 2025 1:56:14 PM (Elapsed Time: 0.09 seconds)
Exporting OCSWITRS Crashes Hot Spots layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [353]:
# Save the project
aprx.save()

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">3.25. Regression Map Layers</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Open Map View</h3>

Close all open maps, open the regression map and set it as the active map

In [354]:
# Close all previous map views
aprx.closeViews()

# Open the regression map view
mapRegression.openView()

# set the main map as active map
map = aprx.activeMap

Remove all layers from the active map

In [355]:
# Remove all the layers from the map
for lyr in mapRegression.listLayers():
    if not lyr.isBasemapLayer:
        print(f"Removing layer: {lyr.name}")
        mapRegression.removeLayer(lyr)

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Add Layers to Map</h3>

Add the feature classes as layers to the map (in order, as the first layer goes to the bottom of the contents)

In [356]:
# Add the data layers to the map
mapRegressionLyrBoundaries = mapRegression.addDataFromPath(boundaries)
mapRegressionLyrCities = mapRegression.addDataFromPath(cities)
mapRegressionLyrCensusBlocks = mapRegression.addDataFromPath(censusBlocks)
mapRegressionLyrRoads = mapRegression.addDataFromPath(roads)

# Set layer visibility on the map
mapRegressionLyrBoundaries.visible = False
mapRegressionLyrCities.visible = False
mapRegressionLyrCensusBlocks.visible = False
mapRegressionLyrRoads.visible = False

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer Symbology</h3>

Define symbology for each of the map layers. The symbology is predefined in the project's template layer folders.

- Roads layer

In [357]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRegressionLyrRoads,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Roads.lyrx"),
    symbology_fields = [["VALUE_FIELD", "roadCat", "roadCat"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:15 PM
Succeeded at Thursday, February 27, 2025 1:56:15 PM (Elapsed Time: 0.06 seconds)


- Census Blocks layer

In [358]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRegressionLyrCensusBlocks,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Census Blocks.lyrx"),
    symbology_fields = [["VALUE_FIELD", "populationDensity", "populationDensity"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:16 PM
Succeeded at Thursday, February 27, 2025 1:56:16 PM (Elapsed Time: 0.06 seconds)


- Cities layer

In [359]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRegressionLyrCities,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Cities.lyrx"),
    symbology_fields = [["VALUE_FIELD", "cityPopDens", "cityPopDens"]],
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:16 PM
Succeeded at Thursday, February 27, 2025 1:56:16 PM (Elapsed Time: 0.11 seconds)


- Boundaries layer

In [360]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = mapRegressionLyrBoundaries,
    in_symbology_layer = os.path.join(layersTemplates, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)
print(arcpy.GetMessages())

Start Time: Thursday, February 27, 2025 1:56:16 PM
Succeeded at Thursday, February 27, 2025 1:56:16 PM (Elapsed Time: 0.04 seconds)


<h3 style="font-weight:bold; color:lime; padding-left: 50px">Layer CIM Operations</h3>

Generate CIM JSON configuration for layers

In [361]:
# Get CIM definitions for the regression map layers
mapRegressionCimBoundaries = mapRegressionLyrBoundaries.getDefinition("V3")
mapRegressionCimCities = mapRegressionLyrCities.getDefinition("V3")
mapRegressionCimCensusBlocks = mapRegressionLyrCensusBlocks.getDefinition("V3")
mapRegressionCimRoads = mapRegressionLyrRoads.getDefinition("V3")

Set symbology headings and update CIM definitions for layers

In [362]:
# Set the layer headings
mapRegressionCimBoundaries.renderer.heading = "Boundaries"
mapRegressionCimCities.renderer.heading = "Cities"
mapRegressionCimCensusBlocks.renderer.heading = "Census Blocks"
mapRegressionCimRoads.renderer.heading = "Roads"

# Update the map layer definitions
mapRegressionLyrBoundaries.setDefinition(mapRegressionCimBoundaries)
mapRegressionLyrCities.setDefinition(mapRegressionCimCities)
mapRegressionLyrCensusBlocks.setDefinition(mapRegressionCimCensusBlocks)
mapRegressionLyrRoads.setDefinition(mapRegressionCimRoads)

Update the CIM defintion for the regression map

In [363]:
cimRegression = mapRegression.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Export Map and Map Layers</h3>

Update the mapx file with the new layers

In [364]:
# Update the victims map mapx file
exportCim("map", mapRegression, "regression")

Exporting regression map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:16 PM
Succeeded at Thursday, February 27, 2025 1:56:16 PM (Elapsed Time: 0.04 seconds)
Exporting regression map to JSON...



Export map layers as CIM JSON `.lyrx` files to the layers folder directory of the project.

In [365]:
# Export the layers to JSON
for l in mapRegression.listLayers():
    if not l.isBasemapLayer:
        exportCim("layer", l, l.name)

Exporting OCSWITRS Roads layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.08 seconds)
Exporting OCSWITRS Roads layer to JSON...

Exporting OCSWITRS Census Blocks layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.07 seconds)
Exporting OCSWITRS Census Blocks layer to JSON...

Exporting OCSWITRS Cities layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.06 seconds)
Exporting OCSWITRS Cities layer to JSON...

Exporting OCSWITRS Boundaries layer to LYRX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.05 seconds)
Exporting OCSWITRS Boundaries layer to JSON...



<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [366]:
# Save the project
aprx.save()

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">4. Map CIM and Exporting</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">4.1. Map CIM Processing</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Get Map CIM</h3>

For each map, obtain the map CIM object

In [367]:
# Get the CIM definitions for the OCSWIRS maps
cimCollisions = mapCollisions.getDefinition("V3")
cimCrashes = mapCollisions.getDefinition("V3")
cimParties = mapCollisions.getDefinition("V3")
cimVictims = mapVictims.getDefinition("V3")
cimInjuries = mapInjuries.getDefinition("V3")
cimFatalities = mapFatalities.getDefinition("V3")
cimFhs100m1km = mapFhs100m1km.getDefinition("V3")
cimFhs150m2km = mapFhs150m2km.getDefinition("V3")
cimFhs100m5km = mapFhs100m5km.getDefinition("V3")
cimFhsRoads500ft = mapFhsRoads500ft.getDefinition("V3")
cimOhsRoads500ft = mapOhsRoads500ft.getDefinition("V3")
cimRoadCrashes = mapRoadCrashes.getDefinition("V3")
cimRoadHotspots = mapRoadHotspots.getDefinition("V3")
cimRoadBuffers = mapRoadBuffers.getDefinition("V3")
cimRoadSegments = mapRoadSegments.getDefinition("V3")
cimRoads = mapRoads.getDefinition("V3")
cimPointFhs = mapPointFhs.getDefinition("V3")
cimPointOhs = mapPointOhs.getDefinition("V3")
cimPopDens = mapPopDens.getDefinition("V3")
cimHouDens = mapHouDens.getDefinition("V3")
cimAreaCities = mapAreaCities.getDefinition("V3")
cimAreaBlocks = mapAreaBlocks.getDefinition("V3")
cimSummaries = mapSummaries.getDefinition("V3")
cimAnalysis = mapAnalysis.getDefinition("V3")
cimRegression = mapRegression.getDefinition("V3")

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Use Service Layer IDs</h3>

Change the map properties to allow assignment of unique numeric IDs for sharing web layers

In [368]:
# Collisions Map
cimCollisions.useServiceLayerIDs = True
mapCollisions.setDefinition(cimCollisions)
# Crashes Map
cimCrashes.useServiceLayerIDs = True
mapCrashes.setDefinition(cimCrashes)
# Parties Map
cimParties.useServiceLayerIDs = True
mapParties.setDefinition(cimParties)
# Victims Map
cimVictims.useServiceLayerIDs = True
mapVictims.setDefinition(cimVictims)
# Injuries Map
cimInjuries.useServiceLayerIDs = True
mapInjuries.setDefinition(cimInjuries)
# Fatalities Map
cimFatalities.useServiceLayerIDs = True
mapFatalities.setDefinition(cimFatalities)
# FHS 100m 1km Map
cimFhs100m1km.useServiceLayerIDs = True
mapFhs100m1km.setDefinition(cimFhs100m1km)
# FHS 150m 2km Map
cimFhs150m2km.useServiceLayerIDs = True
mapFhs150m2km.setDefinition(cimFhs150m2km)
# FHS 100m 5km Map
cimFhs100m5km.useServiceLayerIDs = True
mapFhs100m5km.setDefinition(cimFhs100m5km)
# FHS Roads 500ft Map
cimFhsRoads500ft.useServiceLayerIDs = True
mapFhsRoads500ft.setDefinition(cimFhsRoads500ft)
# OHS Roads 500ft Map
cimOhsRoads500ft.useServiceLayerIDs = True
mapOhsRoads500ft.setDefinition(cimOhsRoads500ft)
# Road Crashes Map
cimRoadCrashes.useServiceLayerIDs = True
mapRoadCrashes.setDefinition(cimRoadCrashes)
# Road Hotspots Map
cimRoadHotspots.useServiceLayerIDs = True
mapRoadHotspots.setDefinition(cimRoadHotspots)
# Road Buffers Map
cimRoadBuffers.useServiceLayerIDs = True
mapRoadBuffers.setDefinition(cimRoadBuffers)
# Road Segments Map
cimRoadSegments.useServiceLayerIDs = True
mapRoadSegments.setDefinition(cimRoadSegments)
# Roads Map
cimRoads.useServiceLayerIDs = True
mapRoads.setDefinition(cimRoads)
# Point FHS Map
cimPointFhs.useServiceLayerIDs = True
mapPointFhs.setDefinition(cimPointFhs)
# Point OHS Map
cimPointOhs.useServiceLayerIDs = True
mapPointOhs.setDefinition(cimPointOhs)
# Population Density Map
cimPopDens.useServiceLayerIDs = True
mapPopDens.setDefinition(cimPopDens)
# Housing Density Map
cimHouDens.useServiceLayerIDs = True
mapHouDens.setDefinition(cimHouDens)
# Area Cities Map
cimAreaCities.useServiceLayerIDs = True
mapAreaCities.setDefinition(cimAreaCities)
# Area Blocks Map
cimAreaBlocks.useServiceLayerIDs = True
mapAreaBlocks.setDefinition(cimAreaBlocks)
# Summaries Map
cimSummaries.useServiceLayerIDs = True
mapSummaries.setDefinition(cimSummaries)
# Analysis Map
cimAnalysis.useServiceLayerIDs = True
mapAnalysis.setDefinition(cimAnalysis)
# Regression Map
cimRegression.useServiceLayerIDs = True
mapRegression.setDefinition(cimRegression)

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">4.2. Export Maps to JSON</h2>

Export maps to mapx CIM JSON files

In [369]:
for m in aprx.listMaps():
    print(f"Exporting {m.name} map...")
    exportCim("map", m, m.name)

Exporting analysis map...
Exporting analysis map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.05 seconds)
Exporting analysis map to JSON...

Exporting areaBlocks map...
Exporting areaBlocks map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.05 seconds)
Exporting areaBlocks map to JSON...

Exporting areaCities map...
Exporting areaCities map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.05 seconds)
Exporting areaCities map to JSON...

Exporting collisions map...
Exporting collisions map to MAPX...
Start Time: Thursday, February 27, 2025 1:56:17 PM
Succeeded at Thursday, February 27, 2025 1:56:17 PM (Elapsed Time: 0.05 seconds)
Exporting collisions map to JSON...

Exporting crashes map...
Exporting crashes map to MAPX...
Start Time: Thur

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Save Project</h3>

In [370]:
# Save the project
aprx.save()

<div style = "background-color:indigo"><center>
<h1 style="font-weight:bold; color:goldenrod; border-top: 2px solid goldenrod; border-bottom: 2px solid goldenrod; padding-top: 5px; padding-bottom: 10px">End of Script</h1>
</center></div>